In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Function to search backward for the first preceding row with 'Lick detected' in the 'Event' column

def search_backwards(df, start_index, target_value, target_column='Event'):
    subset = df.loc[:start_index-1]
    reversed_subset = subset.iloc[::-1]

    # Check if the target value is present in the reversed subset
    if target_value in reversed_subset[target_column].values:
        
        # Find the index of the first occurrence of the target value
        target_index = reversed_subset.index[reversed_subset[target_column] == target_value][0]
        
        return df.loc[target_index]['Arduino_Timestamps']
    
    else:
        return np.nan  # Return an empty DataFrame if the target value is not found

In [ ]:
genotype_dict = {'15-1C-2': 'Nos1',
                 '15-1C-3': 'Nos1',
                 '15-1C-4': 'Nos1',
                 '15-1C-5': 'Nos1',
                 'S9-4B-1R':'Slc30a3',
                 'S9-4B-1L':'Slc30a3',
                 'S9-4B-2R':'Slc30a3',
                 'JAX-10F-2R': 'ZI-SC',
                 'JAX-10F-2L': 'ZI-PRN',
                 'JAX-11F-2R': 'ZI-SC',
                 'JAX-11F-NE2': 'ZI-PRN',
                 'JAX-11M-NE' : 'ZI-PRN',
                 'JAX-11M-1R': 'ZI-SC',
                 'JAX-11M-2L': 'ZI-PRN',
                 'JAX-11M-2R': 'ZI-SC'}

In [ ]:
#Use named, discrete colors to create a custom colormap
cm = ['magenta', 'darkturquoise', 'dimgrey', 'yellow']

    
#Designate which Cell Types you want to plot
ctlist = ['ZI-PRN', 'ZI-SC', 'Nos1', 'Slc30a3']

ctcmdict = dict(zip(ctlist, cm))

# Construct the full dataset dataframe (dfFullSet)

## Note - Need to adjust the directory and file.find( ) lines to process the files of interest

In [ ]:
##Create a single DataFrame to hold all mouse and all session data

directory = '/Volumes/ryanKast/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/AlignedCsvs/'

dflist = []

for root, dirs, files in os.walk(directory):
    
    for file in files:
        print(file)
        
        #Process the properly formatted Arduino/Bonsai CSV outputs
        
        if file.find('TTLaligned') > 0 :
            df = pd.read_csv(directory + file).copy()
            #df = df.rename(columns = {0:'Event', 1: 'Info', 2: 'Arduino_Timestamps'})
            df.insert(0,'Trial Number', df['Event'].str.extract('(Trial.{3})')[0])
            df['Trial Number'] = df['Trial Number'].str.extract(r'Trial (\d+)', expand=False).astype(int, errors = 'ignore')
            df['Trial Number'].ffill(inplace = True)
            df['Mouse'] = file[:file.find('_')]
            df['Day'] = file[file.find('_') + 1:file.find('_TTL')]
            dflist.append(df)
            
                  
dfFullSet = pd.concat(dflist).reset_index(drop = True)
dfFullSet.reset_index(inplace = True)
dfFullSet = dfFullSet.drop(['index'], axis = 1)


#Remove events that occured before the first trial start
dfFullSet = dfFullSet.dropna(subset = ['Trial Number'])

dfFullSet['Trial Number'] = dfFullSet['Trial Number'].astype(int, errors = 'ignore')

#Organize the columns in the order that you want
dfFullSet = dfFullSet[['Day',
 'Mouse',
 'Trial Number',
 'Event',
 'Info',
 'Input',
 'Arduino_Timestamps', 
 'Arduino IEI', 
 'Inper IEI',
 'IEI Diff',
 'Inper_Timestamps', 
 'Guppy Timestamps (s)']]

#Sort the Rows Chronologically
dfFullSet = dfFullSet.sort_values(by = ['Mouse', 'Day', 'Trial Number', 'Arduino_Timestamps']).reset_index(drop = True)

##os.path.basename(os.path.dirname(os.path.dirname(file_path))) This will take you one directory further back

In [ ]:
dfFullSet

In [ ]:
# Create lists to assign additional columns holding data points of interest related to trials to the dfFullSet

sessiontime = []
trialstate = []
trialtime = []
activeport = []
trialoutcome = []
loomtrial = []
pastloomtime = []
loomtrialsback = []
troubletuple = []
trialended = False

for (mouse, day), group in dfFullSet.groupby(['Mouse', 'Day']):
    
    LoomTrial = False
    count = 0
    LoomTime = 0
    LoomTrialPastCount = -1
    TrialStart = 0
    port = 0

    for i in group['Trial Number'].unique().tolist():
        troubletuple.append((mouse,day,i))
        if LoomTrial:
            LoomTrialPastCount = 1
                
        if (LoomTrial == False) & (LoomTrialPastCount >= 0):
            LoomTrialPastCount += 1
            
        sessionstart = group['Arduino_Timestamps'].iloc[0]
        temp = group[group['Trial Number'] == i]
        
        if len(temp[temp['Event'].str.contains('Presented during')]) > 0: 
            LoomTrial = True
            LoomTime = temp[temp['Event'].str.contains('Presented during')]['Arduino_Timestamps'].iloc[0]
            LoomTrialPastCount = 0
            
        else:
            LoomTrial = False
            
        try: 
            trialcomplete = len(temp[temp['Event'].str.contains("ending")])
        except KeyError:
            trialcomplete = 0
            
        if trialcomplete > 0:
            outcome = temp[temp['Event'].str.contains("ending")]['Info'].iloc[0]
            
        else:
            outcome = 'trial incomplete'
                                   
        for n in range(len(temp)):
            
            if temp['Event'].iloc[n].find('started') >= 0:
                TrialStart = temp[temp['Event'].str.contains('started')]['Arduino_Timestamps'].iloc[0]
                port = temp['Info'].iloc[n][7]
                trialtime.append(TrialStart - TrialStart)
                
                if LoomTime != 0:
                    pastloomtime.append(TrialStart - LoomTime)
                else:
                    pastloomtime.append('NaN')
                    
                trialstate.append(1)
                trialended = False
                count += 1
            

            elif temp['Event'].iloc[n].find('ending') >= 0:

                trialstate.append(1)
                trialtime.append(temp['Arduino_Timestamps'].iloc[n] - TrialStart)
                if LoomTime != 0:
                    pastloomtime.append(temp['Arduino_Timestamps'].iloc[n] - LoomTime)
                else:
                    pastloomtime.append('NaN')
                trialended = True

            else:

                if trialended:
                    trialstate.append(0)

                else:
                    trialstate.append(1)

                trialtime.append(temp['Arduino_Timestamps'].iloc[n] - TrialStart)
                
                if LoomTime != 0:
                    pastloomtime.append(temp['Arduino_Timestamps'].iloc[n] - LoomTime)
                else:
                    pastloomtime.append('NaN')
                
            activeport.append(port)
            trialoutcome.append(outcome)
            sessiontime.append(temp['Arduino_Timestamps'].iloc[n] - sessionstart)
            loomtrial.append(LoomTrial)
            loomtrialsback.append(LoomTrialPastCount)
            

In [ ]:
#Assign the new columns to dfFullSet

dfFullSet['Session Time (ms)'] = sessiontime
dfFullSet['Trial Time (ms)'] = trialtime
dfFullSet['Trial State'] = trialstate
dfFullSet['Active Port'] = activeport
dfFullSet['Loom Trial'] = loomtrial
dfFullSet['Trials Since Loom'] = loomtrialsback
dfFullSet['Time Since Loom (ms)'] = pastloomtime
dfFullSet['Trial Outcome'] = trialoutcome
dfFullSet['On Target Lick'] = dfFullSet[dfFullSet['Event'] == 'Lick detected']['Info'].str[-1] == dfFullSet[dfFullSet['Event'] == 'Lick detected']['Active Port']
dfFullSet['Preceding Interlick Interval (ms)'] = dfFullSet[(dfFullSet['Mouse'] == dfFullSet['Mouse'].shift()) & (dfFullSet['Day'] == dfFullSet['Day'].shift()) & (dfFullSet['Event'] == 'Lick detected')]['Arduino_Timestamps'].diff()
dfFullSet['Instantaneous Lick Frequency (Hz)'] = 1000 / dfFullSet['Preceding Interlick Interval (ms)']
dfFullSet.replace({' reward earned': 'reward earned',
                   ' no reward earned': 'no reward earned'},
                 inplace = True)

trialoutcomedig = []
for i in range(len(dfFullSet)):
    if dfFullSet['Trial Outcome'].iloc[i] == 'reward earned':
        trialoutcomedig.append(1)
    else:
        trialoutcomedig.append(0)
dfFullSet['Digitized Trial Outcome'] = trialoutcomedig

In [ ]:
loomnumlist = []
for i in range(len(dfFullSet)):
    if dfFullSet.loc[i]['Event'].find('Presented') != -1:
        num = dfFullSet.loc[i]['Event']
        num = int(num[num.find(' '):num.find(' P')])
        loomnumlist.append(num)
    else:
        loomnumlist.append(np.nan)
        
dfFullSet['Session Loom Number'] = loomnumlist  

In [ ]:
sessionloomlist = []

for i in range(len(dfFullSet)):
    if dfFullSet.iloc[i]['Loom Trial'] == True:
        day = dfFullSet.iloc[i]['Day']
        mouse = dfFullSet.iloc[i]['Mouse']
        trial = dfFullSet.iloc[i]['Trial Number']
        
        temp = dfFullSet[(dfFullSet['Day'] == day) &
                         (dfFullSet['Mouse'] == mouse) &
                         (dfFullSet['Trial Number'] == trial)]
        num = temp[temp['Event'].str.contains('Presented')]
        if len(num) > 0:
            num = num.iloc[0]['Event']
            num = int(num[num.find(' '):num.find(' P')])
        else:
            num = 0
        sessionloomlist.append(num)
    else:
        sessionloomlist.append(0)
        
dfFullSet['Session Loom Number'] = sessionloomlist  

In [ ]:
projectionlist = []
colorlist = []
for mouse in dfFullSet['Mouse'].unique().tolist():
    print(mouse)
    if mouse.find('L') > 0:
        projectionlist.append('ZI-PRN')
        colorlist.append('magenta')
        
    elif mouse.find('R') > 0:
        projectionlist.append('ZI-SC')
        colorlist.append('darkturqoise')

projectdict = dict(zip(dfFullSet['Mouse'].unique().tolist(),projectionlist))
projcmap = dict(zip(dfFullSet['Mouse'].unique().tolist(),colorlist))

In [ ]:
projectionlist = []
colorlist = []
for mouse in dfFullSet['Mouse'].unique().tolist():
    print(mouse)
    if mouse.find('L') > 0:
        projectionlist.append('ZI-PRN')
        colorlist.append('purple')
        
    elif mouse.find('R') > 0:
        projectionlist.append('ZI-SC')
        colorlist.append('teal')

project2dict = dict(zip(dfFullSet['Mouse'].unique().tolist(),projectionlist))
proj2cmap = dict(zip(dfFullSet['Mouse'].unique().tolist(),colorlist))

In [ ]:
ProjectionType = []
for i in range(len(dfFullSet)):
    ProjectionType.append(projectdict[dfFullSet.iloc[i]['Mouse']])
dfFullSet.insert(2, 'Projection Type', ProjectionType)    

In [ ]:
dfFullSet['Info']=dfFullSet['Info'].astype(str,errors = 'ignore')

In [ ]:
dfFullSet['Guppy TS Session Lick Bout Start'] = np.nan
dfFullSet['Guppy TS Session Lick Bout End'] = np.nan

In [ ]:
dfFullSet

# Construct the Trial-level DataFrame (trialdf)

In [ ]:
trialdf = pd.DataFrame()

#determine if the mouse licked the target spout before the trial cutoff (30 seconds)

trialdf['Active Port'] = dfFullSet.groupby(['Mouse', 'Projection Type', 'Day', 'Trial Number'], group_keys=False)['Active Port'].apply(lambda x: x.iloc[0])
trialdf['Trial Immediately Preloaded'] = dfFullSet[dfFullSet["Event"] == "Reward preloaded"].groupby(['Mouse', 'Projection Type', 'Day', 'Trial Number'], group_keys=False).last()['Arduino_Timestamps'] > 0
trialdf['30s Trial Success'] = dfFullSet[dfFullSet["On Target Lick"] == True].groupby(['Mouse', 'Projection Type', 'Day', 'Trial Number'], group_keys=False)["Trial Time (ms)"].min() < 30000
trialdf['Loom Trial'] = dfFullSet.groupby(['Mouse', 'Projection Type', 'Day', 'Trial Number'], group_keys=False)['Loom Trial'].apply(lambda x: x.iloc[0])
trialdf['Session Loom Number'] = dfFullSet.groupby(['Mouse', 'Projection Type', 'Day', 'Trial Number'], group_keys=False)['Session Loom Number'].apply(lambda x: x.iloc[0])
trialdf['Successful Response Time'] = dfFullSet[(dfFullSet["On Target Lick"] == True) & (dfFullSet["Trial Time (ms)"] > 0)].groupby(['Mouse', 'Projection Type', 'Day', 'Trial Number'], group_keys=False)["Trial Time (ms)"].min()
trialdf['Accurate Choice (First Spout On Target)'] = dfFullSet[dfFullSet['Event'] == 'Lick detected'].groupby(['Mouse', 'Projection Type', 'Day', 'Trial Number'], group_keys=False)['On Target Lick'].apply(lambda x: x.iloc[0])
trialdf['On Target Lick Count'] = dfFullSet[dfFullSet['On Target Lick'] == True].groupby(['Mouse', 'Projection Type', 'Day', 'Trial Number'], group_keys=False)['Event'].count()
trialdf['Off Target Lick Count'] = dfFullSet[dfFullSet['On Target Lick'] == False].groupby(['Mouse', 'Projection Type', 'Day', 'Trial Number'], group_keys=False)['Event'].count()
trialdf["Reward Delivery Time"] = dfFullSet[dfFullSet["Event"].str.contains("Milk")].groupby(['Mouse', 'Projection Type', 'Day', 'Trial Number'], group_keys=False)["Trial Time (ms)"].apply(lambda x: x.iloc[0])
trialdf['Trial Outcome'] = dfFullSet[dfFullSet['Event'].str.contains('ending')].groupby(['Mouse', 'Projection Type', 'Day', 'Trial Number'], group_keys=False)['Trial Outcome'].apply(lambda x: x.iloc[0])
trialdf['Trials Since Loom'] = dfFullSet.groupby(['Mouse', 'Projection Type', 'Day', 'Trial Number'], group_keys=False)['Trials Since Loom'].apply(lambda x: x.iloc[0])
trialdf['Digitized Trial Outcome'] = dfFullSet[dfFullSet['Event'].str.contains('ending')].groupby(['Mouse', 'Projection Type', 'Day', 'Trial Number'], group_keys=False)['Digitized Trial Outcome'].apply(lambda x: x.iloc[0])

In [ ]:
trialdf['30s Trial Success'].fillna(False, inplace = True)
trialdf['Trial Immediately Preloaded'].fillna(False, inplace = True)

In [ ]:
trialdf.sort_values(by = ['Mouse', 'Day','Trial Number'], inplace = True)
trialdf = trialdf.reset_index()

In [ ]:
#Give a value for non responses (100 seconds)
trialdf['Successful Response Time'].fillna(100000, inplace = True)

In [ ]:
premature_successfulresponse = []

for i in range(len(trialdf)):
    if trialdf.iloc[i]['Successful Response Time'] < 513:
        premature_successfulresponse.append(True)
    else:
        premature_successfulresponse.append(False)

trialdf['Premature Lick'] = premature_successfulresponse

In [ ]:
volumelist = []
strvolumelist = []
triallist = []

for i in range(len(trialdf)):
    if trialdf['Trial Outcome'].iloc[i] == 'reward earned':
        day = trialdf['Day'].iloc[i]
        mouse = trialdf['Mouse'].iloc[i]
        
        #print(mouse)
        trialnum = trialdf['Trial Number'].iloc[i]
        triallist.append((mouse,day,trialnum))
        temp = dfFullSet[(dfFullSet['Mouse'] == mouse) &
                  (dfFullSet['Day'] == day) &
                  (dfFullSet['Trial Number'] == trialnum) & (dfFullSet['Event'].str.contains('Milk'))]
        if len(temp)> 0:
            strvol = temp.iloc[0]['Event']
            volume = int(re.search(r'\d+', strvol).group())
            volumelist.append(volume)
            strvolumelist.append(strvol)
        else:
            volumelist.append(0)
    else:
        volumelist.append(0)
        strvolumelist.append('Nan')

trialdf['Volumes'] = volumelist
trialdf['Volumes'] = trialdf['Volumes'].astype(int, errors = 'ignore')

# Create DataFrames describing individual Lick Bouts (Interlick Interval <= 1s)

In [ ]:
DATA_A = trialdf[(trialdf['Day'] != 'Day4') & (trialdf['Trials Since Loom'] < 6)].groupby(['Mouse', 'Day', 'Trials Since Loom', 'Active Port'])

successrate = DATA_A['Digitized Trial Outcome'].sum() / DATA_A['Digitized Trial Outcome'].count()
session_successrate = successrate.reset_index()
session_successrate = session_successrate.rename(columns = {'Digitized Trial Outcome': 'Success Rate'})

SessionResponseTimes = DATA_A['Successful Response Time'].mean()
SessionResponseTimes = SessionResponseTimes.reset_index()

In [ ]:
#Add Information related to the lickbouts (IEI < 1000ms) in each trial to dfFullSet
DATA_B = dfFullSet.groupby(['Mouse', 'Day', 'Trial Number'])

lickboutlist = []

for group_name, group_df in DATA_B:
    count = 0
    for i in range(len(group_df)):
        if group_df.iloc[i]['Event'] == 'Lick detected':
            if (group_df.iloc[i]['Preceding Interlick Interval (ms)'] < 1000) & (group_df.iloc[i]['Preceding Interlick Interval (ms)'] > 0):
                if count == 0:
                    count += 1
                lickboutlist.append(count)
            else:
                count += 1
                lickboutlist.append(count)
        else:
            lickboutlist.append('NaN')
                   
dfFullSet['Trial Lick Bout Number'] = lickboutlist

dfFullSet['Trial Lick Bout Number'] = dfFullSet['Trial Lick Bout Number'].astype(int, errors = 'ignore')

In [ ]:
#Add Information related to the lickbouts (IEI < 1000ms) tallied over each entire session to dfFullSet
DATA_C = dfFullSet.groupby(['Mouse', 'Day'])

sessionlickboutlist = []

for group_name, group_df in DATA_C:
    count = 0
    for i in range(len(group_df)):
        if group_df.iloc[i]['Event'] == 'Lick detected':
            if (group_df.iloc[i]['Preceding Interlick Interval (ms)'] < 1000) & (group_df.iloc[i]['Preceding Interlick Interval (ms)'] > 0):
                if count == 0:
                    count += 1
                sessionlickboutlist.append(count)
            else:
                count += 1
                sessionlickboutlist.append(count)
        else:
            sessionlickboutlist.append('NaN')
        
                   
dfFullSet['Session Lick Bout Number'] = sessionlickboutlist
dfFullSet['Session Lick Bout Number'] = dfFullSet['Session Lick Bout Number'].astype(int, errors = 'ignore')

In [ ]:
#Add Information related to the lickepcohs (IEI < 3000ms) tallied over each entire session to dfFullSet
sessionlickepochlist = []

for group_name, group_df in DATA_C:
    count = 0
    for i in range(len(group_df)):
        if group_df.iloc[i]['Event'] == 'Lick detected':
            if (group_df.iloc[i]['Preceding Interlick Interval (ms)'] < 3000) & (group_df.iloc[i]['Preceding Interlick Interval (ms)'] > 0):
                if count == 0:
                    count += 1
                sessionlickepochlist.append(count)
            else:
                count += 1
                sessionlickepochlist.append(count)
        else:
            sessionlickepochlist.append('NaN')
        
                   
dfFullSet['Session Lick Epoch Number'] = sessionlickepochlist
dfFullSet['Session Lick Epoch Number'] = dfFullSet['Session Lick Epoch Number'].astype(int, errors = 'ignore')

In [ ]:
# Create lists to assign additional columns holding data points of interest related to 
# lickbouts in each session
loom_lickbout = []
loom_lickboutsback = []
licks_afterloom_onset_bout = []
time_since_lastlick = []
trial_numberlist = []
trial_statelist = []
ontargetlist = []
volumelist = []

#Group the data based on the Mouse, Projection Type, Day, and Active Port
for (mouse, PT, day, AP), group in dfFullSet.groupby(['Mouse', 'Projection Type', 'Day', 'Active Port']):
    
    BoutsBack = -1
    loombout = False
    
    for i in group['Session Lick Bout Number'].unique().tolist():
        
        if i != 'NaN':
            
            temp = group[group['Session Lick Bout Number'] == i]
            trialnumber = temp.iloc[0]['Trial Number']
            trialstate = temp.iloc[0]['Trial State']
            recentvolume = trialdf[(trialdf['Mouse'] == mouse) & (trialdf['Day'] == day) & (trialdf['Trial Number'] == trialnumber)]['Volumes'].iloc[0]
            ontarget = temp.iloc[0]['On Target Lick']
            first_index = temp.index[0]
            last_index = temp.index[-1]
            
            lastlicktime = group.loc[first_index]['Arduino_Timestamps'] - search_backwards(group, first_index, 'Lick detected')  
            
            temp1 = dfFullSet.iloc[first_index:last_index+1]
           
            if len(temp1[temp1['Event'].str.contains('Presented')]) > 0:
                loombout = True
                BoutsBack = 0
                one = temp1.index[0]
                end = temp1.index[-1]
                loomindex = temp1[temp1['Event'].str.contains('Presented')].index[0]
                temp2 = temp1.iloc[loomindex-one:end-one]
                licks = temp2[temp2['Event'] == 'Lick detected']
                licks_after_loom = len(licks)

            else:
                loombout = False
                BoutsBack += 1
                licks_after_loom = np.nan
                
            licks_afterloom_onset_bout.append(licks_after_loom)
            loom_lickboutsback.append(BoutsBack)
            loom_lickbout.append(loombout)
            time_since_lastlick.append(lastlicktime)
            trial_numberlist.append(trialnumber)
            trial_statelist.append(trialstate)
            ontargetlist.append(ontarget)
            volumelist.append(recentvolume)

In [ ]:
sessionlickboutdf = pd.DataFrame()
sessionlickboutdf['Session Loom Number'] = dfFullSet[dfFullSet['Event'] == 'Lick detected'].groupby(['Mouse', 'Projection Type', 'Day', 'Active Port', 'Session Lick Bout Number'])['Session Loom Number'].apply(lambda x: x.iloc[0])
sessionlickboutdf['Licks in Bout'] = dfFullSet[dfFullSet['Event'] == 'Lick detected'].groupby(['Mouse', 'Projection Type', 'Day', 'Active Port', 'Session Lick Bout Number'], group_keys=False)['Event'].count()
sessionlickboutdf['Max Bout Frequency (Hz)'] = dfFullSet[dfFullSet['Event'] == 'Lick detected'].groupby(['Mouse', 'Projection Type', 'Day', 'Active Port', 'Session Lick Bout Number'], group_keys=False)['Instantaneous Lick Frequency (Hz)'].max()
sessionlickboutdf['Mean Bout Frequency (Hz)'] = dfFullSet[dfFullSet['Event'] == 'Lick detected'].groupby(['Mouse', 'Projection Type', 'Day', 'Active Port', 'Session Lick Bout Number'], group_keys=False)['Instantaneous Lick Frequency (Hz)'].mean()
sessionlickboutdf['Max Bout Interlick Interval (ms)'] = dfFullSet[(dfFullSet['Event'] == 'Lick detected') & (dfFullSet['Preceding Interlick Interval (ms)'] < 5000)].groupby(['Mouse', 'Projection Type', 'Day', 'Active Port', 'Session Lick Bout Number'], group_keys=False)['Preceding Interlick Interval (ms)'].max()
sessionlickboutdf['Lick Bout Duration (ms)'] = dfFullSet[dfFullSet['Event'] == 'Lick detected'].groupby(['Mouse', 'Projection Type', 'Day', 'Active Port', 'Session Lick Bout Number'], group_keys=False)['Arduino_Timestamps'].apply(lambda x: x.max() - x.min())
sessionlickboutdf['Trial Lick Bout Number'] = dfFullSet[dfFullSet['Event'] == 'Lick detected'].groupby(['Mouse', 'Projection Type', 'Day', 'Active Port', 'Session Lick Bout Number'], group_keys=False)['Trial Lick Bout Number'].min()
sessionlickboutdf['Trial Number at Start of Lick Bout'] = trial_numberlist
sessionlickboutdf['Trial State at Start of Lick Bout'] = trial_statelist
sessionlickboutdf['On Target Lick Bout'] = ontargetlist
sessionlickboutdf['Most Recent Volume'] = volumelist
sessionlickboutdf['Lick Bout with Loom'] = loom_lickbout
sessionlickboutdf['Licks in Bout after Loom Onset'] = licks_afterloom_onset_bout
sessionlickboutdf['Lick Bouts Since Last Loom'] = loom_lickboutsback
sessionlickboutdf['Time since last lick (ms)'] = time_since_lastlick
sessionlickboutdf = sessionlickboutdf.sort_values(['Mouse', 'Day', 'Session Lick Bout Number']).reset_index()

In [ ]:
%%time
guppytslist = []

for (mouse,day,AP,lickbout), group in sessionlickboutdf.groupby(['Mouse', 'Day', 'Active Port', 'Session Lick Bout Number'], sort = False):
    
    condition = (dfFullSet['Mouse'] == mouse) & (dfFullSet['Day'] == day) & (dfFullSet['Session Lick Bout Number'] == lickbout)
    Gtimestamp = dfFullSet.loc[condition, 'Guppy Timestamps (s)'].iloc[0]
    guppytslist.append(Gtimestamp) 
    

In [ ]:
%%time

end_guppytslist = []

for (mouse,day,AP,lickbout), group in sessionlickboutdf.groupby(['Mouse', 'Day', 'Active Port', 'Session Lick Bout Number'], sort = False):
    
    Gtimestamp = dfFullSet.query("Mouse == @mouse and Day == @day and `Session Lick Bout Number` == @lickbout").iloc[-1]['Guppy Timestamps (s)']
    end_guppytslist.append(Gtimestamp) 

In [ ]:
sessionlickboutdf['Start Guppy Timestamp (s)'] = guppytslist
sessionlickboutdf['End Guppy Timestamp (s)'] = end_guppytslist

In [ ]:
bins = [3,6,12,25,100]
sessionlickboutdf['Binned Licks in Bout'] = pd.cut(sessionlickboutdf['Licks in Bout'], bins=bins, labels =bins[1:])
sessionlickboutdf['Binned Licks in Bout'].value_counts()

[i for i in merged_df.columns.tolist() if i not in dfFullSet.columns.tolist()]

In [ ]:

duration_bins = [0,1000,2000,4000,60000]

sessionlickboutdf['Binned Lick Bout Duration'] = pd.cut(sessionlickboutdf['Lick Bout Duration (ms)'], bins=duration_bins, labels =duration_bins[1:])
sessionlickboutdf['Binned Lick Bout Duration'].value_counts()

# Create DataFrames describing individual Lick Epochs (Interlick interval <= 3s)

In [ ]:
%%time

# Create lists to assign additional columns holding data points of interest related to 
# lick epochs in each session
loom_lickepoch = []
loom_lickepochsback = []
licks_afterloom_onset_epoch = []
bouts_afterloom_onset_epoch = []
time_since_lastlick_epoch = []
trial_numberlist_epoch = []
trial_statelist_epoch = []
ontargetlist_epoch = []
volumelist_epoch = []

#Group the data based on the Mouse, Projection Type, and Day
for (mouse, PT, day), group in dfFullSet.groupby(['Mouse', 'Projection Type', 'Day']):
    
    BoutsBack = -1
    loombout = False
    
    for i in group['Session Lick Epoch Number'].unique().tolist():
        
        if i != 'NaN':
            
            temp = group[group['Session Lick Epoch Number'] == i]
            trialnumber = temp.iloc[0]['Trial Number']
            trialstate = temp.iloc[0]['Trial State']
            recentvolume = trialdf[(trialdf['Mouse'] == mouse) & (trialdf['Day'] == day) & (trialdf['Trial Number'] == trialnumber)]['Volumes'].iloc[0]
            ontarget = temp.iloc[0]['On Target Lick']
            first_index = temp.index[0]
            last_index = temp.index[-1]

            lastlicktime = group.loc[first_index]['Arduino_Timestamps'] - search_backwards(group, first_index, 'Lick detected')  

            temp1 = dfFullSet.iloc[first_index:last_index+1]

            if len(temp1[temp1['Event'].str.contains('Presented')]) > 0:
                loombout = True
                BoutsBack = 0
                
                one = temp1.index[0] #define the first index for this lick epoch
                end = temp1.index[-1] #define the last index for this lick epoch
                
                loomindex = temp1[temp1['Event'].str.contains('Presented')].index[0] #Extract index of the loom presentation
                
                temp2 = temp1.iloc[loomindex-one:end-one] #slice the dataframe from loom onset to end of the lick epoch
                licks = temp2[temp2['Event'] == 'Lick detected']
                licks_after_loom = len(licks)
                bouts_after_loom = temp2[temp2['Event'] == 'Lick detected']['Session Lick Bout Number'].nunique()

            else:
                loombout = False
                BoutsBack += 1
                licks_after_loom = np.nan
                bouts_after_loom = np.nan

            licks_afterloom_onset_epoch.append(licks_after_loom)
            bouts_afterloom_onset_epoch.append(bouts_after_loom)
            loom_lickepochsback.append(BoutsBack)
            loom_lickepoch.append(loombout)
            time_since_lastlick_epoch.append(lastlicktime)
            trial_numberlist_epoch.append(trialnumber)
            trial_statelist_epoch.append(trialstate)
            ontargetlist_epoch.append(ontarget)
            volumelist_epoch.append(recentvolume)


In [ ]:
sessionlickepochdf = pd.DataFrame()
sessionlickepochdf['Session Loom Number'] = dfFullSet[dfFullSet['Event'] == 'Lick detected'].groupby(['Mouse', 'Projection Type', 'Day', 'Session Lick Epoch Number'])['Session Loom Number'].apply(lambda x: x.iloc[0])
sessionlickepochdf['Licks in Epoch'] = dfFullSet[dfFullSet['Event'] == 'Lick detected'].groupby(['Mouse', 'Projection Type', 'Day', 'Session Lick Epoch Number'], group_keys=False)['Event'].count()
sessionlickepochdf['Lick Bouts in Epoch'] = dfFullSet[dfFullSet['Event'] == 'Lick detected'].groupby(['Mouse', 'Projection Type', 'Day', 'Session Lick Epoch Number'], group_keys=False)['Session Lick Bout Number'].nunique()
sessionlickepochdf['Max Epoch Frequency (Hz)'] = dfFullSet[dfFullSet['Event'] == 'Lick detected'].groupby(['Mouse', 'Projection Type', 'Day', 'Session Lick Epoch Number'], group_keys=False)['Instantaneous Lick Frequency (Hz)'].max()
sessionlickepochdf['Mean Epoch Frequency (Hz)'] = dfFullSet[dfFullSet['Event'] == 'Lick detected'].groupby(['Mouse', 'Projection Type', 'Day', 'Session Lick Epoch Number'], group_keys=False)['Instantaneous Lick Frequency (Hz)'].mean()
sessionlickepochdf['Lick Epoch Duration (ms)'] = dfFullSet[dfFullSet['Event'] == 'Lick detected'].groupby(['Mouse', 'Projection Type', 'Day', 'Session Lick Epoch Number'], group_keys=False)['Arduino_Timestamps'].apply(lambda x: x.max() - x.min())
sessionlickepochdf['Trial Number at Start of Lick Epoch'] = trial_numberlist_epoch
sessionlickepochdf['Trial State at Start of Lick Epoch'] = trial_statelist_epoch
sessionlickepochdf['On Target Lick Bout'] = ontargetlist_epoch
sessionlickepochdf['Most Recent Volume'] = volumelist_epoch
sessionlickepochdf['Lick Epoch with Loom'] = loom_lickepoch
sessionlickepochdf['Licks in Epoch after Loom Onset'] = licks_afterloom_onset_epoch
sessionlickepochdf['Bouts in Epoch after Loom Onset'] = bouts_afterloom_onset_epoch
sessionlickepochdf['Lick Bouts Since Last Loom'] = loom_lickepochsback
sessionlickepochdf['Time since last lick (ms)'] = time_since_lastlick_epoch
sessionlickepochdf = sessionlickepochdf.sort_values(['Mouse', 'Day', 'Session Lick Epoch Number']).reset_index()
sessionlickepochdf = sessionlickepochdf.reset_index(drop = True)

In [ ]:
%%time

guppytslist_epoch = []

for (mouse,day,lickepoch), group in sessionlickepochdf.groupby(['Mouse', 'Day', 'Session Lick Epoch Number'], sort = False):
    
    Gtimestamp = dfFullSet.query("Mouse == @mouse and Day == @day and `Session Lick Epoch Number` == @lickepoch").iloc[0]['Guppy Timestamps (s)']
    guppytslist_epoch.append(Gtimestamp) 

In [ ]:
sessionlickepochdf['First Lick Guppy Timestamp (s)'] = guppytslist_epoch

%%time
## Fast Vectorization not reliable, perhaps due to .first() call
mergeDF = pd.merge(sessionlickepochdf, dfFullSet, how='inner',
                         left_on=['Mouse', 'Day', 'Session Lick Epoch Number'],
                         right_on=['Mouse', 'Day', 'Session Lick Epoch Number'])

guppytslist_epoch_fast =  mergeDF.groupby(['Mouse', 'Day', 'Session Lick Epoch Number'])['Guppy Timestamps (s)'].first().tolist()


# Vectorized searching of pandas dataframes, but giving unreliable results
merged_df = pd.merge(sessionlickboutdf, dfFullSet, how='inner',
                     left_on=['Mouse', 'Day', 'Active Port', 'Session Lick Bout Number'],
                     right_on=['Mouse', 'Day', 'Active Port', 'Session Lick Bout Number'])

# Filter the merged dataframe based on conditions
condition = merged_df['Guppy Timestamps (s)'].notna()
guppytslist2 = merged_df_end.groupby(['Mouse', 'Day', 'Active Port', 'Session Lick Bout Number'])['Guppy Timestamps (s)'].iloc[0].tolist()

guppytslist3 = merged_df_end.groupby(['Mouse', 'Day', 'Active Port', 'Session Lick Bout Number'])['Guppy Timestamps (s)'].first().tolist()

# Vectorized searching of pandas dataframes, but giving unreliable results

# Merge dataframes based on conditions
merged_df_end = pd.merge(sessionlickboutdf, dfFullSet, how='inner',
                         left_on=['Mouse', 'Day', 'Active Port', 'Session Lick Bout Number'],
                         right_on=['Mouse', 'Day', 'Active Port', 'Session Lick Bout Number'])

# Extract the last element for each group
end_guppytslist2 = merged_df_end.groupby(['Mouse', 'Day', 'Active Port', 'Session Lick Bout Number'])['Guppy Timestamps (s)'].last().tolist()

end_guppytslist3 = merged_df_end.groupby(['Mouse', 'Day', 'Active Port', 'Session Lick Bout Number'])['Guppy Timestamps (s)'].iloc[-1].tolist()


## Attempted to make this marginally faster, but given weird data output, I reverted for at least the time being.

%%time
# Initialize lists
boutwmostlicks_trial_FAST = []
first_bwml_ts_FAST = []
last_bwml_ts_FAST = []

# Group the data based on the Mouse, Day, and Trial Number
for (mouse, day, trial), group in dfFullSet.groupby(['Mouse', 'Day', 'Trial Number']):
    
    if 'Lick detected' in group['Event'].values:
        lick_bout_numbers = group.loc[group['Event'] == 'Lick detected', 'Session Lick Bout Number']
        most_licked_bout = lick_bout_numbers.mode().iloc[0]

        boutwmostlicks_trial_FAST.append(most_licked_bout)

        # Use vectorized operations to find the first and last timestamps
        first_bwml_ts_FAST.append(group.loc[group['Session Lick Bout Number'] == most_licked_bout, 'Guppy Timestamps (s)'].iloc[0])
        last_bwml_ts_FAST.append(group.loc[group['Session Lick Bout Number'] == most_licked_bout, 'Guppy Timestamps (s)'].iloc[-1])
        
    else:
        boutwmostlicks_trial_FAST.append(np.nan)
        first_bwml_ts_FAST.append(np.nan)
        last_bwml_ts_FAST.append(np.nan)


# Add Lick Bout information to the trialdf

In [ ]:
%%time
#Add Guppy Timestamps For the First and Last Licks 
#of the first Lick Bout with at least 3 licks in Each Trial
#the trialdf
threelickbout_trial = []
first_tlb_ts = []
last_tlb_ts = []

for (mouse,day,trial), group in dfFullSet.groupby(['Mouse', 'Day', 'Trial Number']):
    print(mouse,day,trial)
    for slbn in group['Session Lick Bout Number'].unique().tolist(): 
        if len(group[(group['On Target Lick'] == True) & (group['Session Lick Bout Number'] == slbn)]) >= 3:
            threelickbout_trial.append(slbn)
            first_tlb_ts.append(group.loc[group['Session Lick Bout Number'] == slbn, 'Guppy Timestamps (s)'].iloc[0])
            last_tlb_ts.append(group.loc[group['Session Lick Bout Number'] == slbn, 'Guppy Timestamps (s)'].iloc[-1])
            break
    
        elif len(group[(group['On Target Lick'] == True) & (group['Session Lick Bout Number'] == group['Session Lick Bout Number'].mode().iloc[0])]) < 3:
            threelickbout_trial.append(np.nan)
            first_tlb_ts.append(np.nan)
            last_tlb_ts.append(np.nan)
            break

In [ ]:
%%time
#Add Guppy Timestamps For the First and Last Licks 
#of the Lick Bout with the most licks in Each Trial to 
#the trialdf

boutwmostlicks_trial = []
first_bwml_ts = []
last_bwml_ts = []

# Also add just the first on target lick of the trial
first_lick_ts = []

for (mouse,day,trial), group in dfFullSet.groupby(['Mouse', 'Day', 'Trial Number']):
    
    if len(group[group['On Target Lick'] == True]) > 0:
        n = group[group['On Target Lick'] == True]['Session Lick Bout Number'].mode().iloc[0]
        boutwmostlicks_trial.append(n)
        first_bwml_ts.append(group[group['Session Lick Bout Number'] == n].iloc[0]['Guppy Timestamps (s)'])
        last_bwml_ts.append(group[group['Session Lick Bout Number'] == n].iloc[-1]['Guppy Timestamps (s)'])
        first_lick_ts.append(group[group['On Target Lick'] == True].iloc[0]['Guppy Timestamps (s)'])
        
    else:
        boutwmostlicks_trial.append(np.nan)
        first_bwml_ts.append(np.nan)
        last_bwml_ts.append(np.nan)
        first_lick_ts.append(np.nan)
        

In [ ]:
trialdf['Guppy TS First 3 Lick Bout in Trial (First Lick)'] = first_tlb_ts
trialdf['Guppy TS First 3 Lick Bout in Trial (Last Lick)'] = last_tlb_ts
trialdf['First Session Lick Bout with At Least 3 Licks in Current Trial'] = threelickbout_trial
trialdf['Guppy TS Bout w Most Licks (First Lick)'] = first_bwml_ts
trialdf['Guppy TS Bout w Most Licks (Last Lick)'] = last_bwml_ts
trialdf['Session Lick Bout with Most Licks in Current Trial'] = boutwmostlicks_trial
trialdf['First Lick Guppy TS'] = first_lick_ts

# Save the Preprocessed Output

 - dfFullSet
 - trialdf
 - sessionlickboutdf
 - sessionlickepochdf

In [ ]:
sessionlickepochdf.to_csv('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/IntermediatePythonOutput/DNDSessionlickepochdf_022924rerunnotebook.csv')
sessionlickboutdf.to_csv('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/IntermediatePythonOutput/DNDSessionlickboutdf_022923rerunnotebook.csv')
trialdf.to_csv('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/IntermediatePythonOutput/DNDtrialdf_022924rerunnotebook.csv')
dfFullSet.to_csv('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/IntermediatePythonOutput/DNDdfFullSet_022923rerunnotebook.csv')


# Start from here with pre-generated input 3-25-2024


In [ ]:
# Mac
dfFullSet = pd.read_csv('/Volumes/ryanKast/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/IntermediatePythonOutput/DNDdfFullSet_022923rerunnotebook.csv')
trialdf = pd.read_csv('/Volumes/ryanKast/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/IntermediatePythonOutput/DNDtrialdf_022924rerunnotebook.csv')                
sessionlickboutdf = pd.read_csv('/Volumes/ryanKast/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/IntermediatePythonOutput/DNDSessionlickboutdf_022923rerunnotebook.csv')  

In [ ]:
# Windows

dfFullSet = pd.read_csv('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/IntermediatePythonOutput/DNDdfFullSet_022923rerunnotebook.csv')
trialdf = pd.read_csv('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/IntermediatePythonOutput/DNDtrialdf_022924rerunnotebook.csv')                
sessionlickboutdf = pd.read_csv('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/IntermediatePythonOutput/DNDSessionlickboutdf_022923rerunnotebook.csv')  

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)                        

In [ ]:
dfMerged = dfFullSet.merge(sessionlickboutdf, how='outer',on=['Mouse','Day', 'Session Lick Bout Number'],suffixes=(None, '_slb'))
dfMerged.shape#.merge(trialdf,how='outer', on=['Mouse','Day','Trial Number'])

# Create Guppy Formatted TTL Files

In [ ]:
dfFullSet[(dfFullSet['Mouse']=='JAX-15F-1L' ) &
         (dfFullSet['Day']== 'Day1') &
         (dfFullSet['Session Lick Bout Number'] == 58 )]

In [ ]:
#This Cell Produced All Of The Lick Related Final Guppy Event Files On 2-29-24 & 10PM EST

#Trial-oriented eventTTLs
#Guppy eventTTL files for the first lick bout in each trial that has at least 3 licks


In [ ]:
#This Cell Produced All Of The Lick Related Final Guppy Event Files On 2-2-24 & 3:54PM EST

#Trial-oriented eventTTLs
#Guppy eventTTL files for the first lick bout in each trial that has at least 3 licks

for (mouse, day, digitizedtrialoutcome), group in dfFullSet.groupby(['Mouse', 'Day', 'Digitized Trial Outcome']):
    df = pd.DataFrame()
    if digitizedtrialoutcome == 1:
        outcome = 'Successful_'
    elif digitizedtrialoutcome == 0:
        outcome = 'Failed_'
    df['timestamps'] = group[group['Event'].str.contains(r'Trial \d+ started')]['Guppy Timestamps (s)']
    
    dst_dir = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs2-2-24/" + day + '/' + mouse + '/'
    os.makedirs(dst_dir, exist_ok=True)
    df.to_csv(dst_dir + outcome + 'TrialStart' + '.csv', index = False)
    

count = 0

for (mouse,day,volume), group in trialdf[~trialdf['Premature Lick']].groupby(['Mouse', 'Day', 'Volumes']):
    volumestr = str(int((volume/100) + 5))
    count += 1
    group.sort_values('Trial Number', inplace = True)
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    df['timestamps'] =  group['Guppy TS First 3 Lick Bout in Trial (First Lick)']
    df2['timestamps'] = group['Guppy TS First 3 Lick Bout in Trial (Last Lick)']
    dst_dir = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs2-2-24/" + day + '/' + mouse + '/'
    os.makedirs(dst_dir, exist_ok=True)
    df[df.notna()].reset_index(drop = True).to_csv(dst_dir + str(volumestr) + 'uLRewardVolume_TrialInitial3lickbout_FirstLick_bothports.csv', index = False)
    df2[df2.notna()].reset_index(drop = True).to_csv(dst_dir + str(volumestr) + 'uLRewardVolume_TrialInitial3lickbout_LastLick_bothports.csv', index = False)


#Lick bout-oriented eventTTLs
#FirstOnTarget
#Guppy eventTTL files for large volume (25uL) lick bouts with or without Loom
for (mouse, day, loom), group in sessionlickboutdf[(sessionlickboutdf['Day'] == 'Day4') & (sessionlickboutdf['Most Recent Volume'] == 2000)].groupby(['Mouse', 'Day', 'Lick Bout with Loom']):
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    group.sort_values('Start Guppy Timestamp (s)', inplace = True)
    if loom:
        value = 'Loom.csv'
    else:
        value = 'NoLoom.csv'
    df['timestamps'] = group[group['On Target Lick Bout']]['Start Guppy Timestamp (s)']
    df2['timestamps'] = group[group['On Target Lick Bout']]['End Guppy Timestamp (s)']

    dst_dir = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs2-2-24/" + day + '/' + mouse + '/'
    #os.makedirs(dst_dir, exist_ok=True)
    df.to_csv(dst_dir + 'FirstLick_25uLReward_' + value, index = False)
    df2.to_csv(dst_dir + 'LastLick_25uLReward_' + value, index = False)
    
#Lick bout-oriented eventTTLs
#FirstOnTarget
#LastOnTarget
#Guppy eventTTL files for lick bouts binned by lick bout duration
preceed_interval = '5000'

for (mouse, day, bins), group in sessionlickboutdf.groupby(['Mouse', 'Day', 'Binned Lick Bout Duration']):
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    group.sort_values('Start Guppy Timestamp (s)', inplace = True)
    
    
    df['timestamps'] = group[(group['On Target Lick Bout']) &
                             #& (group['Active Port'] == activeport)
                        (group['Time since last lick (ms)'] >  int(preceed_interval))]['Start Guppy Timestamp (s)']
    df2['timestamps'] = group[(group['On Target Lick Bout']) 
                             #& (group['Active Port'] == activeport)
                       & (group['Time since last lick (ms)'] >  int(preceed_interval))]['End Guppy Timestamp (s)']
    
    
    dst_dir = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs2-2-24/" + day + '/' + mouse + '/'
    os.makedirs(dst_dir, exist_ok=True)
    df.to_csv(dst_dir + 'FirstOnTarget_BothPorts_' + str(bins) + 'mslickboutbin_' + preceed_interval +'msIEI.csv', index = False)                                                                                             
    df2.to_csv(dst_dir + 'LastOnTarget_BothPorts_' + str(bins) + 'mslickboutbin_' + preceed_interval +'msIEI.csv', index = False)                                                                                             
                                                                                                       

#Lick bout-oriented eventTTLs
#FirstOnTarget
#LastOnTarget
#Guppy eventTTL files for lick bouts binned by lick bout duration
preceed_interval = '5000'

for (mouse, day), group in sessionlickboutdf.groupby(['Mouse', 'Day']):
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    group.sort_values('Start Guppy Timestamp (s)', inplace = True)
    df['timestamps'] = group[(group['On Target Lick Bout']) &
                             #& (group['Active Port'] == activeport)
                        (group['Time since last lick (ms)'] >  int(preceed_interval))]['Start Guppy Timestamp (s)']
    df2['timestamps'] = group[(group['On Target Lick Bout'])
                             #& (group['Active Port'] == activeport)
                       & (group['Time since last lick (ms)'] >  int(preceed_interval))]['End Guppy Timestamp (s)']

    dst_dir = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs2-2-24/" + day + '/' + mouse + '/'
    os.makedirs(dst_dir, exist_ok=True)
    df.to_csv(dst_dir + 'FirstOnTarget_BothPorts_AllLickBouts' + preceed_interval +'msIEI.csv', index = False)                                                                                             
    df2.to_csv(dst_dir + 'LastOnTarget_BothPorts_AllLickBouts' + preceed_interval +'msIEI.csv', index = False)                                                                                             
 

In [ ]:
sessionlickboutdf['Binned Lick Bout Duration'].value_counts()

In [ ]:
sessionlickboutdf[(sessionlickboutdf['Mouse'] == 'JAX-15F-2L') 
                  & (sessionlickboutdf['Day'] == 'Day3') 
                  & (sessionlickboutdf['Binned Lick Bout Duration'] == 4000)
                  & (sessionlickboutdf['Time since last lick (ms)'] > 5000)
                  & (sessionlickboutdf['Trial Lick Bout Number'] == 1)]

In [ ]:
for  (day,mouse), group in dfFullSet[dfFullSet['Loom Trial']].groupby(['Day','Mouse']):
    df = pd.DataFrame()
    df['timestamps'] = group[group['Event'].str.contains('Presented')]['Guppy Timestamps (s)']
    
    dst_dir = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs2-2-24/" + day + '/' + mouse + '/'
    df.to_csv(dst_dir + 'AllLooms.csv', index = False) 
    


# End Guppy TTL Generation 2-2-24

In [ ]:
# Add ZI-SC and ZI-PRN suffices to the end of the 410nm and 470nm fluorescence data .csvs

sourcepath = 'Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs2-2-24/'

search_pattern = os.path.join(sourcepath, '**/**')
files = glob.glob(search_pattern, recursive=True)

# Copy each found file to the destination directory
for file in files:
    if file.find('nmguppyformat') != -1:
        if (file.find('ZI-SC') == -1) & (file.find('ZI-PRN') == -1):
            _, day, mouse, filename = file.split(os.sep)[-7:]
        
            if mouse.find('R') != -1:

                os.rename(file, file.replace('.csv', '-ZI-SC.csv'))
            
            elif mouse.find('L') != -1:

                os.rename(file, file.replace('.csv', '-ZI-PRN.csv'))
                



for (mouse, day, digitizedtrialoutcome), group in dfFullSet.groupby(['Mouse', 'Day', 'Digitized Trial Outcome']):
    df = pd.DataFrame()
    if digitizedtrialoutcome == 1:
        outcome = 'Successful_'
    elif digitizedtrialoutcome == 0:
        outcome = 'Failed_'
    df['timestamps'] = group[group['Event'].str.contains(r'Trial \d+ started')]['Guppy Timestamps (s)']
    
    dst_dir = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs/" + day + '/' + mouse + '/'
    os.makedirs(dst_dir, exist_ok=True)
    df.to_csv(dst_dir + outcome + 'TrialStart' + '.csv', index = False)
    

#Trial-oriented eventTTLs
#Guppy eventTTL files for the first lick bout in each trial that has at least 3 licks
count = 0

for (mouse,day,volume), group in trialdf[~trialdf['Premature Lick']].groupby(['Mouse', 'Day', 'Volumes']):
    volumestr = str(int((volume/100) + 5))
    count += 1
    group.sort_values('Trial Number', inplace = True)
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    df['timestamps'] =  group['Guppy TS First 3 Lick Bout in Trial (First Lick)']
    df2['timestamps'] = group['Guppy TS First 3 Lick Bout in Trial (Last Lick)']
    dst_dir = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs/" + day + '/' + mouse + '/'
    os.makedirs(dst_dir, exist_ok=True)
    df[df.notna()].reset_index(drop = True).to_csv(dst_dir + str(volumestr) + 'uLRewardVolume_TrialInitial3lickbout_FirstLick_bothports.csv', index = False)
    df2[df2.notna()].reset_index(drop = True).to_csv(dst_dir + str(volumestr) + 'uLRewardVolume_TrialInitial3lickbout_LastLick_bothports.csv', index = False)

#Lick bout-oriented eventTTLs
#FirstOnTarget
#Guppy eventTTL files for lick bouts binned by lick bout duration


for (mouse, day, loom), group in sessionlickboutdf[(sessionlickboutdf['Day'] == 'Day4') & (sessionlickboutdf['Most Recent Volume'] == 2000)].groupby(['Mouse', 'Day', 'Lick Bout with Loom']):
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    group.sort_values('Start Guppy Timestamp (s)', inplace = True)
    if loom:
        value = 'Loom.csv'
    else:
        value = 'NoLoom.csv'
    df['timestamps'] = group[group['On Target Lick Bout']]['Start Guppy Timestamp (s)']
    df2['timestamps'] = group[group['On Target Lick Bout']]['End Guppy Timestamp (s)']

    dst_dir = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs/" + day + '/' + mouse + '/'
    #os.makedirs(dst_dir, exist_ok=True)
    df.to_csv(dst_dir + 'FirstLick_25uLReward_' + value, index = False)
    df2.to_csv(dst_dir + 'LastLick_25uLReward_' + value, index = False)

                                                                                                       

#Lick bout-oriented eventTTLs
#FirstOnTarget
#LastOnTarget
#Guppy eventTTL files for lick bouts binned by lick bout duration
preceed_interval = '5000'

for (mouse, day, bins), group in sessionlickboutdf.groupby(['Mouse', 'Day', 'Binned Lick Bout Duration']):
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    group.sort_values('Start Guppy Timestamp (s)', inplace = True)
    
    df['timestamps'] = group[(group['On Target Lick Bout']) &
                             #& (group['Active Port'] == activeport)
                        (group['Time since last lick (ms)'] >  int(preceed_interval))]['Start Guppy Timestamp (s)']
    df2['timestamps'] = group[(group['On Target Lick Bout']) 
                             #& (group['Active Port'] == activeport)
                       & (group['Time since last lick (ms)'] >  int(preceed_interval))]['End Guppy Timestamp (s)']
    
    
    dst_dir = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs/" + day + '/' + mouse + '/'
    os.makedirs(dst_dir, exist_ok=True)
    df.to_csv(dst_dir + 'FirstOnTarget_BothPorts_' + str(bins) + 'mslickboutbin_' + preceed_interval +'msIEI.csv', index = False)                                                                                             
    df2.to_csv(dst_dir + 'LastOnTarget_BothPorts_' + str(bins) + 'mslickboutbin_' + preceed_interval +'msIEI.csv', index = False)                                                                                             
                                                                                                       

#Lick bout-oriented eventTTLs
#FirstOnTarget
#LastOnTarget
#Guppy eventTTL files for lick bouts binned by lick bout duration
preceed_interval = '5000'

for (mouse, day), group in sessionlickboutdf.groupby(['Mouse', 'Day']):
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    group.sort_values('Start Guppy Timestamp (s)', inplace = True)
    df['timestamps'] = group[(group['On Target Lick Bout']) &
                             #& (group['Active Port'] == activeport)
                        (group['Time since last lick (ms)'] >  int(preceed_interval))]['Start Guppy Timestamp (s)']
    df2['timestamps'] = group[(group['On Target Lick Bout'])
                             #& (group['Active Port'] == activeport)
                       & (group['Time since last lick (ms)'] >  int(preceed_interval))]['End Guppy Timestamp (s)']

    dst_dir = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs/" + day + '/' + mouse + '/'
    os.makedirs(dst_dir, exist_ok=True)
    df.to_csv(dst_dir + 'FirstOnTarget_BothPorts_AllLickBouts' + preceed_interval +'msIEI.csv', index = False)                                                                                             
    df2.to_csv(dst_dir + 'LastOnTarget_BothPorts_AllLickBouts' + preceed_interval +'msIEI.csv', index = False)                                                                                             
                                                                                                       

#Lick bout-oriented eventTTLs
#Guppy eventTTL files for lick bouts binned by total number of licks in bout
#Spout 4
preceed_interval = '5000'

for (mouse, day, bins), group in sessionlickboutdf.groupby(['Mouse', 'Day', 'Binned Licks in Bout']):
    df = pd.DataFrame()
    df['timestamps'] = group[(group['On Target Lick Bout']) & 
                             (group['Active Port'] == '4') & 
                             (group['Time since last lick (ms)'] >  int(preceed_interval))]['Start Guppy Timestamp (s)']

    dst_dir = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs/" + day + '/' + mouse + '/'
    os.makedirs(dst_dir, exist_ok=True)
    df.to_csv(dst_dir + 'FirstOnTarget_Spout4_' + str(bins) + 'binlicks' + preceed_interval +'msIEI.csv', index = False)                                                                                             
                                                                                                       

#Lick bout-oriented eventTTLs
#Guppy eventTTL files for lick bouts binned by total number of licks in bout
#Spout 1
preceed_interval = '5000'

for (mouse, day, bins), group in sessionlickboutdf.groupby(['Mouse', 'Day', 'Binned Licks in Bout']):
    df = pd.DataFrame()
    df['timestamps'] = group[(group['On Target Lick Bout']) & 
                             (group['Active Port'] == '1') & 
                             (group['Time since last lick (ms)'] >  int(preceed_interval))]['Start Guppy Timestamp (s)']

    dst_dir = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs/" + day + '/' + mouse + '/'
    os.makedirs(dst_dir, exist_ok=True)
    df.to_csv(dst_dir + 'FirstOnTarget_Spout1_' + str(bins) + 'binlicks' + preceed_interval +'msIEI.csv', index = False)                                                                                             
                                                                                                       

#Lick bout-oriented eventTTLs
#Guppy eventTTL files for lick bouts binned by total number of licks in bout
#Both Spouts
preceed_interval = '5000'

for (mouse, day, bins), group in sessionlickboutdf.groupby(['Mouse', 'Day', 'Binned Licks in Bout']):
    df = pd.DataFrame()
    df['timestamps'] = group[(group['On Target Lick Bout']) & 
                             #(group['Active Port'] == '4') & 
                             (group['Time since last lick (ms)'] >  int(preceed_interval))]['Start Guppy Timestamp (s)']

    dst_dir = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs/" + day + '/' + mouse + '/'
    os.makedirs(dst_dir, exist_ok=True)
    df.to_csv(dst_dir + 'FirstOnTarget_BothPorts_' + str(bins) + 'binlicks' + preceed_interval +'msIEI.csv', index = False)                                                                                             
                                                                                                       

#Lick bout-oriented eventTTLs
#Guppy eventTTL files for lick bouts binned by total number of licks in bout
#Both Spouts
preceed_interval = '5000'

for (mouse, day, bins), group in sessionlickboutdf.groupby(['Mouse', 'Day', 'Binned Licks in Bout']):
    df = pd.DataFrame()
    df['timestamps'] = group[(group['On Target Lick Bout']) & 
                             #(group['Active Port'] == '4') & 
                             (group['Time since last lick (ms)'] >  int(preceed_interval))]['End Guppy Timestamp (s)']

    dst_dir = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs/" + day + '/' + mouse + '/'
    os.makedirs(dst_dir, exist_ok=True)
    df.to_csv(dst_dir + 'LastOnTarget_BothPorts_' + str(bins) + 'binlicks' + preceed_interval +'msIEI.csv', index = False)                                                                                             
                                                                                                       

sortvalue = 'Lick Bout Duration (ms)'
preceed_interval = '10000'

for (mouse,day), group in sessionlickboutdf.groupby(['Mouse', 'Day']):
    df = pd.DataFrame()
    df['timestamps'] = group[(group['Active Port'] == '4') 
               & (group['On Target Lick Bout']) 
               & (group['Time since last lick (ms)'] >  int(preceed_interval))].sort_values(sortvalue)['Guppy Timestamp (s)']
  
    dst_dir = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs/" + day + '/' + mouse + '/'
    os.makedirs(dst_dir, exist_ok=True)
    df.to_csv(dst_dir + 'OnTarget-' + sortvalue + '-S4-Ascend_'+ preceed_interval +'msIEI.csv', index = False)

In [ ]:
#sna = storenames_a
sna = [i.replace('.csv','') for i in os.listdir('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs2-2-24/Day2\JAX-15F-1L') if i.find('.csv') > 0]

In [ ]:
sna2 = [i.replace('.csv','') for i in os.listdir('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs2-2-24/Day4\JAX-15F-1L') if i.find('.csv') > 0]

sna3 = [i.replace('.csv','') for i in os.listdir('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs2-2-24/Day4\JAX-17M-1L') if i.find('.csv') > 0]

In [ ]:
sna = sna + sna2 + sna3

In [ ]:
sna2

In [ ]:
sndict = {}
sndict['LastOnTarget_BothPorts_AllLickBouts5000msIEI'] = 'AllBouts_LastLick_BothPorts'
sndict['FirstOnTarget_BothPorts_AllLickBouts5000msIEI'] = 'AllBouts_FirstLick_BothPorts'
sndict['AllLooms'] =  'AllLooms'
sndict['Looms_Escape'] = 'LoomsEscape'
sndict['Looms_NoEscape'] = 'LoomsNoEscape'
sndict['Escape_onset'] = 'Escape Onset'
sndict['Shelter_arrival'] = 'Shelter Arrival'
sndict['Failed_TrialStart'] = 'TrialStart_Failed'
sndict['Successful_TrialStart'] = 'TrialStart_Success'
sndict['data-000-sig-410nmguppyformat-ZI-PRN'] =  'control_ZI-PRN'
sndict['data-000-sig-470nmguppyformat-ZI-PRN'] = 'signal_ZI-PRN'
sndict['data-000-sig-410nmguppyformat-ZI-SC'] =  'control_ZI-SC'
sndict['data-000-sig-470nmguppyformat-ZI-SC'] = 'signal_ZI-SC'

for i in sna:
    
    if i not in sndict.keys():
        
        if i.find('10000msIEI') > 1:
            IEI = '10sIEI'
        elif i.find('5000msIEI') > 1:
            IEI = '5sIEI'
            
        if i.find('Last') != -1:
            
            if i.find('binlick') != -1:
                sndict[i]  = 'LastLick-' + i[i.find('_') + 1 : i.find('lick')] # + IEI
                
            elif i.find('mslick') != -1:
                sndict[i] =  'LastLick-' + i[i.find('_') + 1 : i.find('lick')] # + IEI
                
            elif i.find('RewardVolume') != -1:
                sndict[i] = 'BothPorts_LastLick-' + i[: i.find('_')]
                
            elif i.find('_Loom') != -1:
                sndict[i] = 'BothPorts_LastLick-25uL_Loom'
                
            elif i.find('_NoLoom') != -1:
                sndict[i] = 'BothPorts_LastLick-25uL_NoLoom'
                
        elif i.find('First') != -1:
            
            if i.find('binlick') != -1:
                sndict[i] = 'FirstLick-' + i[i.find('_') + 1 : i.find('lick')] # + IEI
        
            elif i.find('mslick') != -1:
                sndict[i] = 'FirstLick-' + i[i.find('_') + 1 : i.find('lick')] # + IEI
            
            elif i.find('RewardVolume') != -1:
                sndict[i] = 'BothPorts_FirstLick-' + i[: i.find('_')]
                
            elif i.find('_Loom') != -1:
                sndict[i] = 'BothPorts_FirstLick-25uL_Loom'

            elif i.find('_NoLoom') != -1:
                sndict[i] = 'BothPorts_FirstLick-25uL_NoLoom'

In [ ]:
sndict = {}
sndict['LastOnTarget_BothPorts_AllLickBouts5000msIEI'] = 'AllBouts_Last'
sndict['FirstOnTarget_BothPorts_AllLickBouts5000msIEI'] = 'AllBouts_First'
sndict['5second_freezes'] = 'Shelter_Freeze'
sndict['AllLooms'] =  'AllLooms'
sndict['Looms_Escape'] = 'LoomsEscape'
sndict['Looms_NoEscape'] = 'LoomsNoEscape'
sndict['Escape_onset'] = 'Escape Onset'
sndict['Shelter_arrival'] = 'Shelter Arrival'
sndict['Failed_TrialStart'] = 'TrialStart_Failed'
sndict['Successful_TrialStart'] = 'TrialStart_Success'
sndict['PostFreeze_mediummovement'] = 'Immobility_End'
sndict['PostFreeze_smallmovement'] = 'Freeze_End'
sndict['Shelter_Arrival_ShelterCam'] = 'ShelterArrival_Best'
sndict['Shelter_Exit'] = 'PostLoom_ShelterExit'
sndict['data-000-sig-410nmguppyformat-ZI-PRN'] =  'control_ZI-PRN'
sndict['data-000-sig-470nmguppyformat-ZI-PRN'] = 'signal_ZI-PRN'
sndict['data-000-sig-410nmguppyformat-ZI-SC'] =  'control_ZI-SC'
sndict['data-000-sig-470nmguppyformat-ZI-SC'] = 'signal_ZI-SC'

for i in sna:
    
    if i not in sndict.keys():
        
        if i.find('10000msIEI') > 1:
            IEI = '10sIEI'
        elif i.find('5000msIEI') > 1:
            IEI = '5sIEI'
            
        if i.find('Last') != -1:
            
            if i.find('binlick') != -1:
                sndict[i]  = 'Last-' + i[i.find('_') + 1 : i.find('0')] + 's' # + IEI
                
            elif i.find('mslick') != -1:
                sndict[i] =  'Last-' + i[i.find('_') + 1 : i.find('0')] + 's' # + IEI
                
            elif i.find('RewardVolume') != -1:
                sndict[i] = 'Last-' + i[: i.find('_')]
                
            elif i.find('_Loom') != -1:
                sndict[i] = 'Last-25uL_Loom'
                
            elif i.find('_NoLoom') != -1:
                sndict[i] = 'Last-25uL_NoLoom'
                
        elif i.find('BinFreeze') > 1:
                sndict[i] = 'BinFreeze-' + i[:i.find('_')]
                
        elif i.find('First') != -1:
            
            if i.find('binlick') != -1:
                sndict[i] = 'First-' + i[i.find('_') + 1 : i.find('0')] + 's' # + IEI
        
            elif i.find('mslick') != -1:
                sndict[i] = 'First-' + i[i.find('_') + 1 : i.find('0')] + 's' # + IEI
            
            elif i.find('RewardVolume') != -1:
                sndict[i] = 'First-' + i[: i.find('_')]
                
            elif i.find('_Loom') != -1:
                sndict[i] = 'First-25uL_Loom'

            elif i.find('_NoLoom') != -1:
                sndict[i] = 'First-25uL_NoLoom'

In [ ]:
sndict

In [ ]:
#remove empty csvs in the analysis directories

# Define the directory to walk through
directory = "C:/Users/Fading/Code/GuppyFormatCsvs2-2-24/Day4/"

# Walk through the directory
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith('.csv'):  # Check if the file is a CSV file
            file_path = os.path.join(root, file)
            try:
                # Read the CSV file into a DataFrame
                df = pd.read_csv(file_path)
                # Check the number of rows
                num_rows = len(df)
                if num_rows == 0:
                    # Remove the file if there are no rows
                    os.remove(file_path)
                    print(f"File '{file}' removed because it has 0 rows.")
                else:
                    print(f"File '{file}' has {num_rows} rows.")
            except pd.errors.EmptyDataError:
                # Handle the case when the file is empty
                os.remove(file_path)
                print(f"File '{file}' removed because it is empty.")
            except Exception as e:
                print(f"Error processing file '{file}': {e}")


In [ ]:
import shutil
direct = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs2-2-24/Day4/"
dest = "C:/Users/Fading/Code/GuppyFormatCsvs2-2-24/Day4/"
for root,dirs,files in os.walk(direct):
    for file in files:
    
        if ((file.find('Shelter') != -1))| ((file.find('PostFreeze') != -1) & (file.find('movement') != -1) & (file.find('AUC') == -1)):
            mouse = root.split('/')[-1]
            if mouse.find('JAX') != -1:
                print(file)
                print(dest + mouse + '/' + file)
                shutil.copy2(root + '/' + file, dest + mouse + '/' +  file)


In [ ]:
import shutil
direct = "Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs2-2-24/Day4/"
dest = "C:/Users/Fading/Code/GuppyFormatCsvs2-2-24/Day4/"
for root,dirs,files in os.walk(dest):
    for file in files:
    
        if (root.find('output') == -1) & (file.find('BinFreeze') != -1):
            mouse = root.split('/')[-1]

            if mouse.find('JAX') != -1:
                print(root + '/' + file)
                os.remove(root + '/' + file)
                #shutil.copy2(root + '/' + file, dest + mouse + '/' +  file)


In [ ]:
CreateStoresListCSV("C:/Users/Fading/Code/GuppyFormatCsvs2-2-24/Day4/", sndict)

import glob
sourcepath = 'Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/GuppyFormatCsvs2-2-24/'
search_pattern = os.path.join(sourcepath, '**/**.csv')
files = glob.glob(search_pattern, recursive=True)

# Copy each found file to the destination directory
for file in files:
        os.remove(file)
        print(file)

In [ ]:
CreateStoresListCSV("C:/Users/Fading/Code/GuppyFormatCsvs2-2-24/Day4/", sndict)

# Create Guppy Storenames Files

In [ ]:
test_storenames1 = os.listdir('Z:\RKDATA\Behavior\FiberPhotometryData\Analysis\DND_DiscreteAnalysis\GuppyFormatCsvs\Day4\JAX-16F-1L')

In [ ]:
test_storenames2 = os.listdir('Z:\RKDATA\Behavior\FiberPhotometryData\Analysis\DND_DiscreteAnalysis\GuppyFormatCsvs\Day4\JAX-16F-1R')

In [ ]:
test_storenames = test_storenames1 + test_storenames2

In [ ]:
test_storenames = [i for i in set(test_storenames) if i.find('output') == -1]

In [ ]:
test_storenames = [i.replace('.csv', '') for i in test_storenames]

In [ ]:
sndict = {}

In [ ]:
sndict['AllLoomTimestamps'] =  'AllLooms'
sndict['data-000-sig-410nmguppyformat-ZI-PRN'] =  'control_ZI-PRN'
sndict['data-000-sig-470nmguppyformat-ZI-PRN'] = 'signal_ZI-PRN'
sndict['data-000-sig-410nmguppyformat-ZI-SC'] =  'control_ZI-SC'
sndict['data-000-sig-470nmguppyformat-ZI-SC'] = 'signal_ZI-SC'

In [ ]:
def CreateStoresListCSV(data_path, storenamesdict):

    """
        Creates the storesList.csv file and the "output_#" subdirectory that will contain the Guppy output
        
        2 Arguments needed:
            1. data_path - path to the directory containing the guppy formatted photometry data and event ttls
            2. storenamesdict - a dictionary with:
                keys = csv_filenames
                values = storenames
    """
    
    for root, dirs, files in os.walk(data_path):
        
        for filename in dirs:
            
            # Construct the full file path
            filepath = root + '/' + filename
            
            # Split the path by backslashes
            path_parts = filepath.split('/')
            
            if filepath.find('output') == -1:

                # Extract the Day and Mouse information
                try:
                    day = path_parts[-2]
                    mouse = path_parts[-1]
                except IndexError:
                    # Insufficient parts after 'Day' index
                    continue

                print(mouse, day)

                guppycsvs = []

                for file in os.listdir(data_path +  day + '/' + mouse + '/'):
                    if file.find('.csv') != -1:
                        guppycsvs.append(file.replace('.csv', ''))
                storenames = []

                for file in guppycsvs:
                    storenames.append(storenamesdict[file])
                
                if (len(guppycsvs) > 0) & (len(storenames) > 0):
                    df = pd.DataFrame(columns = guppycsvs)
                    new_row = dict(zip(guppycsvs,storenames))
                    df.loc[len(df)] = new_row
                    dst_dir = data_path + day + '\\' + mouse + '\\' + mouse + '_output_1\\'
                    os.makedirs(dst_dir, exist_ok=True)
                    df.to_csv(dst_dir + 'storesList.csv', index = False)

In [ ]:
sessionlickboutdf.columns.tolist()

# Generate CSVs to crop Videos around loom Trials

In [ ]:
for (mouse, day), group in dfFullSet[(dfFullSet['Day'] == 'Day4') &
                                     (dfFullSet['Loom Trial'])].groupby(['Mouse', 'Day']):
    df = pd.DataFrame()
    df2 = pd.DataFrame() #for creating start:stop frame csv that Bonsai will use to split the video into loom trials
    df['timestamps'] = group[group['Event'].str.contains('Presented')]['Guppy Timestamps (s)']
    
    df2['start'] = (df['timestamps'] - 4) * 30
    df2['stop'] =  (df['timestamps'] + 120) * 30
    df2['start'] = df2['start'].astype(int)
    df2['stop'] = df2['stop'].astype(int)
    dst_dir = "Z:/RKDATA/Behavior/FiberPhotometryData/ZIPathway_GCaMP8m/Cohort2/DineAndDash/" + day + '/' + mouse + '/'
    os.makedirs(dst_dir, exist_ok=True)
    #df.to_csv(dst_dir + 'AllLoomTimestamps.csv', index = False)
    df2.to_csv(dst_dir + 'ShelterLoomTrialIndices.csv', index = False, header = False)

## Plotting

In [ ]:
sns.set_theme(style = 'white', 
              font = 'Helvetica',
              font_scale = 2)

afig = plt.figure(figsize = (6,3), dpi =300)

mice = list(project2dict.keys())

mouse_cmap = [proj2cmap[i] for i in mice]


ax = sns.PairGrid(data=session_successrate[(session_successrate['Trials Since Loom'] == -1)],
                  x_vars = 'Day',
                  y_vars = 'Success Rate',
                  hue ='Mouse',
                  hue_order = mice,
                  palette = mouse_cmap,
                  height = 5,
                  )


ax.map(sns.pointplot)
ax.set(ylim=(0, 1.1),
       xlabel = '')


In [ ]:

sns.set_theme(style = 'white', 
              font = 'helvetica',
              font_scale = 1)

sns.set_style('ticks')
plt.figure(figsize = (2,3), dpi =300)

#plt.xticks(rotation=90)


g = sns.pointplot(data=session_successrate[(session_successrate['Mouse'] != 'JAX-15F-1R') &(session_successrate['Trials Since Loom'] == -1)],
                  x = 'Day',
                  y = 'Success Rate',
                  hue = 'Active Port',
                  palette = ['grey','orange']).set(ylim=[-0.05,1.1], xlabel = 'Day',  yticks = [0.0,0.2,0.4,0.6,0.8,1.0])


sns.despine(top=True,right=True)

plt.legend(bbox_to_anchor=(1.05, 1),
           loc='upper left',
           title = 'Active Port',
           borderaxespad=0, 
           title_fontsize = 8,
           fontsize = 8)

plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/SuccessRate_vs_Days_DiffPorts.svg', dpi=300, bbox_inches = 'tight')
plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/SuccessRate_vs_Days_DiffPorts.png', dpi=300, bbox_inches = 'tight')




In [ ]:
sns.set_theme(style = 'white', 
              font = 'helvetica',
              font_scale = 1)

sns.set_style('ticks')
plt.figure(figsize = (2,3), dpi =300)

#plt.xticks(rotation=90)


g = sns.boxplot(data=session_successrate[(session_successrate['Mouse'] != 'JAX-15F-1R') & (session_successrate['Trials Since Loom'] == -1)],
                  x = 'Day',
                  y = 'Success Rate',
                  color = 'grey').set(ylim=[-0.05,1.1], xlabel = 'Day',  yticks = [0.0,0.2,0.4,0.6,0.8,1.0])


sns.despine(top=True,right=True)

#plt.savefig('/Users/ryankast/Documents/FengLab/Manuscripts/ZI_CellTypes/V4/Fig5/SuccessRate_vs_Days.svg', dpi=300, bbox_inches = 'tight')
#plt.savefig('/Users/ryankast/Documents/FengLab/Manuscripts/ZI_CellTypes/V4/Fig5/SuccessRate_vs_Days.png', dpi=300, bbox_inches = 'tight')


In [ ]:
sns.set_theme(style = 'white', 
              font = 'arial',
              font_scale = 1.5)

plt.figure(figsize = (4,4), dpi =300)


mice = list(project2dict.keys())

mouse_cmap = [proj2cmap[i] for i in mice]

#plt.xticks(rotation=90)


sns.pointplot(data=session_successrate[(session_successrate['Trials Since Loom'] == -1)],
                  x = 'Day',
                  y = 'Success Rate',
                  hue ='Mouse',
                  alpha = 0.5,
                  hue_order = mice,
                  palette = mouse_cmap).set(ylim=[0,1.1], xlabel = '')

plt.legend(bbox_to_anchor=(1.05, 1),
           labels = ['ZI-PRN','ZI-SC'],
           loc='upper left', 
           borderaxespad=0, 
           title = 'Projection Type', 
           title_fontsize = 8,
           fontsize = 8)



In [ ]:
plt.figure(figsize = (5,4), dpi=300)

sns.kdeplot(data = sessionlickepochdf[(sessionlickepochdf['Licks in Epoch'] > 3) & 
                                      (sessionlickepochdf['Day'].isin(['Day3', 'Day2']))],
            
            x= 'Lick Epoch Duration (ms)',
            hue = (sessionlickepochdf['Most Recent Volume'] / 100) + 5,
            palette = 'plasma', cut = 0).set(title = 'Licks Epoch Duration (All)\nvs Reward Volume\non Days 2 and 3')

In [ ]:
plt.figure(figsize = (5,4), dpi=300)

sns.kdeplot(data = sessionlickepochdf[(sessionlickepochdf['Licks in Epoch'] > 3) & 
                                      (sessionlickepochdf['Day'].isin(['Day3', 'Day2']))],
            
            x= 'Licks in Epoch',
            hue = (sessionlickepochdf['Most Recent Volume'] / 100) + 5,
            palette = 'plasma', cut = 0).set(title = 'Licks Per Epoch (All)\nvs Reward Volume\non Days 2 and 3')

In [ ]:
sessionlickboutdf.columns.tolist()

In [ ]:
plt.figure(figsize = (5,4), dpi=300)

sns.kdeplot(data = sessionlickboutdf[ (sessionlickboutdf['Licks in Bout'] > 3) & 
                                  (sessionlickboutdf['Day'] == 'Day4') &
                                  (sessionlickboutdf['Most Recent Volume'] == 2000) &
                                  (sessionlickboutdf['Trial Lick Bout Number'] == 1) &
                                  (sessionlickboutdf['Session Loom Number'] < 10)  ], cut = 0,
            x= 'Licks in Bout',
            hue = 'Session Loom Number',
            palette = 'plasma')

sns.despine(top = True, right = True)

In [ ]:
trialdf

In [ ]:
trialdf.info()

In [ ]:
trialdf

In [ ]:
bins = [1,11,21,31,41,51]
trialdf['Binned Trials'] = pd.cut(trialdf['Trial Number'], bins=bins, labels = ['1-10','11-20','21-30','31-40','41-50'])

In [ ]:

sns.set_theme(style = 'white', 
              font = 'helvetica',
              font_scale = 1)

sns.set_style('ticks')

plt.figure(figsize=(3,4), dpi = 300)
ax = sns.pointplot(data = trialdf[(trialdf['30s Trial Success']) & (trialdf['Day'] != 'Day4' ) & (trialdf['Mouse'] != 'JAX-15F-1R')],
                    x = 'Binned Trials', 
                    y = trialdf['Successful Response Time']/ 1000, 
                    hue = 'Day',
                    palette = 'viridis', alpha = 0.5)
                  
ax.set_ylim(0,20)
sns.despine(top=True, right = True)

ax.set_xlabel('Trial Bins')
ax.set_ylabel('Response Time (s)')
ax.set_yticks([0,5,10,15,20])
plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/Reward_ResponseTime_Days_BinnedTrials.png', bbox_inches = 'tight',dpi =300)

In [ ]:
plt.figure(figsize = (5,4), dpi=300)

sns.histplot(data = trialdf[(trialdf['Day'].isin(['Day2','Day3']))],
                x = 'On Target Lick Count',
                hue = 'Volumes',
                palette = 'plasma',
                alpha = 0.3,
                #cut = 0)
            ).set(title = 'Trial On Target Lick Count\nvs Reward Volume\non Days 2 and 3')

sns.despine(top = True, right = True)

plt.legend(bbox_to_anchor=(1.05, 1),
           labels = [25, 20, 15, 10, 5],
           loc='upper left', 
           borderaxespad=0, 
           title = 'Reward Volume (uL)', 
           title_fontsize = 8,
           fontsize = 8)


In [ ]:
#https://stackoverflow.com/questions/18926031/how-to-extract-a-subset-of-a-colormap-as-a-new-colormap-in-matplotlib
    
import matplotlib.colors as colors
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

arr = np.linspace(0, 50, 10).reshape((5, 2))
fig, ax = plt.subplots(ncols=2)

cmap = plt.get_cmap('plasma')
new_cmap = truncate_colormap(cmap, 0, 0.9)
ax[0].imshow(arr, interpolation='nearest', cmap=cmap)
ax[1].imshow(arr, interpolation='nearest', cmap=new_cmap)
plt.show()

In [ ]:
plt.figure(figsize = (5,4), dpi=300)

sns.kdeplot(data = trialdf[(trialdf['Day'].isin(['Day2','Day3']))],
                x = 'On Target Lick Count',
                hue = 'Volumes',
                palette = new_cmap,
                cut = 0).set(title = 'Trial On Target Lick Count\nvs Reward Volume\non Days 2 and 3')

sns.despine(top = True, right = True)

plt.legend(bbox_to_anchor=(1.05, 1),
           labels = [25, 20, 15, 10, 5],
           loc='upper left', 
           borderaxespad=0, 
           title = 'Reward Volume (uL)', 
           title_fontsize = 8,
           fontsize = 8)

plt.savefig('/Users/ryankast/Documents/FengLab/Manuscripts/ZI_CellTypes/V4/Fig5/KDE_OnTargetLickCount_vs_Volume_Days2&3.svg', dpi=300, bbox_inches = 'tight')
plt.savefig('/Users/ryankast/Documents/FengLab/Manuscripts/ZI_CellTypes/V4/Fig5/KDE_OnTargetLickCount_vs_Volume_Days2&3.png', dpi=300, bbox_inches = 'tight')


In [ ]:
plt.figure(figsize = (5,4), dpi=300)

sns.kdeplot(data = trialdf[(trialdf['Day'].isin(['Day4'])) &
                           (trialdf['Volumes'] == 2000)],
                x = 'On Target Lick Count',
                hue = 'Session Loom Number',
                palette = 'plasma',
                cut = 0).set(title = 'Trial On Target Lick Count\nvs Reward Volume\non Days 2 and 3')

sns.despine(top = True, right = True)

plt.legend(bbox_to_anchor=(1.05, 1),
           labels = [25, 20, 15, 10, 5],
           loc='upper left', 
           borderaxespad=0, 
           title = 'Reward Volume (uL)', 
           title_fontsize = 8,
           fontsize = 8)


In [ ]:
plt.figure(figsize = (5,4), dpi=300)

sns.kdeplot(data = sessionlickboutdf[(sessionlickboutdf['Licks in Bout'] >= 5) & 
                                      (sessionlickboutdf['On Target Lick Bout']) &
                                      (sessionlickboutdf['Day'].isin([ 'Day4'])) &
                                      (sessionlickboutdf['Trial Lick Bout Number'].isin([1])) &
                                      (sessionlickboutdf['Most Recent Volume'] == 2000) &
                                     (sessionlickboutdf['Time since last lick (ms)'] > 5000)],
            cut = 0,
            x= 'Licks in Bout',
            hue = 'Lick Bout with Loom').set(title = 'Licks Per Bout\nvs Reward Volume\n Loom vs Non-Loom')

sns.despine(top = True, right = True)

plt.legend(bbox_to_anchor=(1.05, 1),
           loc='upper left',
           labels = ['Loom', 'No Loom'],
           borderaxespad=0, 
           title = 'Reward Volume (uL)', 
           title_fontsize = 8,
           fontsize = 8)

In [ ]:
sessionlickboutdf.columns.tolist()

In [ ]:
plt.figure(figsize = (2,3), dpi=300)

sns.kdeplot(data = sessionlickboutdf[(sessionlickboutdf['Licks in Bout'] >= 3) & 
                                      (sessionlickboutdf['On Target Lick Bout']) &
                                      (sessionlickboutdf['Day'].isin([ 'Day2', 'Day3'])) &
                                      (sessionlickboutdf['Trial Lick Bout Number'].isin([1])) &
                                     (sessionlickboutdf['Time since last lick (ms)'] > 5000)],
            cut = 0,
            x= 'Licks in Bout',
            hue = 'Most Recent Volume',
            palette = new_cmap).set(title = 'Licks Per Bout\nvs Reward Volume\non Days 2 and 3')

sns.despine(top = True, right = True)

plt.legend(bbox_to_anchor=(1.05, 1),
           labels = [25, 20, 15, 10, 5],
           loc='upper left', 
           borderaxespad=0, 
           title = 'Reward Volume (uL)', 
           title_fontsize = 8,
           fontsize = 8)

#plt.savefig('/Users/ryankast/Documents/FengLab/Manuscripts/ZI_CellTypes/V4/Fig5/KDE_LicksPerBout_vs_Volume_EitherPort_Days2&3.svg', dpi=300, bbox_inches = 'tight')
#plt.savefig('/Users/ryankast/Documents/FengLab/Manuscripts/ZI_CellTypes/V4/Fig5/KDE_LicksPerBout_vs_Volume_EitherPort_Days2&3.png', dpi=300, bbox_inches = 'tight')


In [ ]:
plt.figure(figsize = (5,4), dpi=300)

sns.kdeplot(data = sessionlickboutdf[(sessionlickboutdf['Licks in Bout'] >= 3) & 
                                      (sessionlickboutdf['On Target Lick Bout']) &
                                      (sessionlickboutdf['Day'].isin([ 'Day2', 'Day3'])) &
                                      (sessionlickboutdf['Trial Lick Bout Number'].isin([1])) &
                                     (sessionlickboutdf['Active Port'] == '4') &
                                     (sessionlickboutdf['Time since last lick (ms)'] > 5000)],
            cut = 0,
            x= 'Licks in Bout',
            hue = 'Most Recent Volume',
            palette = new_cmap).set(title = 'Licks Per Bout at Port 4\nvs Reward Volume\non Days 2 and 3')

sns.despine(top = True, right = True)

plt.legend(bbox_to_anchor=(1.05, 1),
           labels = [25, 20, 15, 10, 5],
           loc='upper left', 
           borderaxespad=0, 
           title = 'Reward Volume (uL)', 
           title_fontsize = 8,
           fontsize = 8)

plt.savefig('/Users/ryankast/Documents/FengLab/Manuscripts/ZI_CellTypes/V4/Fig5/KDE_LicksPerBout_vs_Volume_Port4_Days2&3.svg', dpi=300, bbox_inches = 'tight')
plt.savefig('/Users/ryankast/Documents/FengLab/Manuscripts/ZI_CellTypes/V4/Fig5/KDE_LicksPerBout_vs_Volume_Port4_Days2&3.png', dpi=300, bbox_inches = 'tight')


In [ ]:
plt.figure(figsize = (2,3), dpi=300)

filtereddf = sessionlickboutdf[(sessionlickboutdf['Licks in Bout'] >= 3) & 
                                      (sessionlickboutdf['On Target Lick Bout']) &
                                      (sessionlickboutdf['Day'].isin([ 'Day2', 'Day3'])) &
                                      (sessionlickboutdf['Trial Lick Bout Number'].isin([1])) &
                                      (sessionlickboutdf['Time since last lick (ms)'] > 5000)]

ax1 = sns.kdeplot(data = filtereddf,
            cut = 0,
            x= filtereddf['Lick Bout Duration (ms)'] / 1000,
            hue = 'Most Recent Volume',
            palette = new_cmap)

ax1.set_title('Duration First Lick Bout of Each Trial\nvs Reward Volume at Port 1\non Days 2 and 3')

sns.despine(top = True, right = True)

plt.legend(bbox_to_anchor=(1.05, 1),
           labels = [25, 20, 15, 10, 5],
           loc='upper left', 
           borderaxespad=0, 
           title = 'Reward Volume (uL)', 
           title_fontsize = 8,
           fontsize = 8)

ax1.set_xticks(range(0,16,2))
               
ax1.set_xlim(0,14)

ax1.set_xlabel('Lick Bout Duration (s)')


plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/KDE_LickBoutDuration_vs_Volume_Port1_Days2&3.svg', dpi=300, bbox_inches = 'tight')
plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/KDE_LickBoutDuration_vs_Volume_Port1_Days2&3.png', dpi=300, bbox_inches = 'tight')


In [ ]:
plt.figure(figsize = (5,4), dpi=300)

sns.kdeplot(data = sessionlickboutdf[(sessionlickboutdf['Licks in Bout'] >= 3) & 
                                      (sessionlickboutdf['On Target Lick Bout']) &
                                      (sessionlickboutdf['Day'].isin([ 'Day2', 'Day3'])) &
                                      (sessionlickboutdf['Trial Lick Bout Number'].isin([1])) &
                                      (sessionlickboutdf['Active Port'] == '4') &
                                      (sessionlickboutdf['Time since last lick (ms)'] > 5000)],
            cut = 0,
            x= 'Lick Bout Duration (ms)',
            hue = 'Most Recent Volume',
            palette = new_cmap).set(title = 'Duration First Lick Bout of Each Trial\nvs Reward Volume at Port 4\non Days 2 and 3')

sns.despine(top = True, right = True)

plt.legend(bbox_to_anchor=(1.05, 1),
           labels = [25, 20, 15, 10, 5],
           loc='upper left', 
           borderaxespad=0, 
           title = 'Reward Volume (uL)', 
           title_fontsize = 8,
           fontsize = 8)

plt.savefig('/Users/ryankast/Documents/FengLab/Manuscripts/ZI_CellTypes/V4/Fig5/KDE_LickBoutDuration_vs_Volume_Port4_Days2&3.svg', dpi=300, bbox_inches = 'tight')
plt.savefig('/Users/ryankast/Documents/FengLab/Manuscripts/ZI_CellTypes/V4/Fig5/KDE_LickBoutDuration_vs_Volume_Port4_Days2&3.png', dpi=300, bbox_inches = 'tight')


In [ ]:
plt.figure(figsize = (5,4), dpi=300)

sns.kdeplot(data = sessionlickboutdf[(sessionlickboutdf['Licks in Bout'] >= 5) & 
                                      (sessionlickboutdf['On Target Lick Bout']) &
                                      (sessionlickboutdf['Day'].isin([ 'Day3'])) &
                                      (sessionlickboutdf['Trial Lick Bout Number'].isin([1])) &
                                      
                                      (sessionlickboutdf['Time since last lick (ms)'] > 5000)],
            cut = 0,
            x= 'Lick Bout Duration (ms)',
            hue = 'Most Recent Volume',
            palette = new_cmap).set(title = 'Duration First Lick Bout of Each Trial\nvs Reward Volume at Either Port\non Days 2 and 3')

sns.despine(top = True, right = True)

plt.legend(bbox_to_anchor=(1.05, 1),
           labels = [25, 20, 15, 10, 5],
           loc='upper left', 
           borderaxespad=0, 
           title = 'Reward Volume (uL)', 
           title_fontsize = 8,
           fontsize = 8)

plt.savefig('/Users/ryankast/Documents/FengLab/Manuscripts/ZI_CellTypes/V4/Fig5/KDE_LickBoutDuration_vs_Volume_EitherPort_Days2&3.svg', dpi=300, bbox_inches = 'tight')
plt.savefig('/Users/ryankast/Documents/FengLab/Manuscripts/ZI_CellTypes/V4/Fig5/KDE_LickBoutDuration_vs_Volume_EitherPort_Days2&3.png', dpi=300, bbox_inches = 'tight')

In [ ]:
plt.figure(figsize = (5,4), dpi=300)

sns.ecdfplot(data = sessionlickboutdf[(sessionlickboutdf['Licks in Bout'] >= 5) & 
                                      (sessionlickboutdf['On Target Lick Bout']) &
                                      (sessionlickboutdf['Day'].isin([ 'Day2', 'Day3'])) &
                                      (sessionlickboutdf['Trial Lick Bout Number'].isin([1])) &
                                      
                                      (sessionlickboutdf['Time since last lick (ms)'] > 5000)],
            x= 'Licks in Bout',
            hue = 'Mouse',
            hue_order = sessionlickboutdf['Mouse'].unique().tolist(),
            palette = proj_palette,
            alpha = 0.5
            ).set(title = 'Cumulative Distributions Licks in Bout per Mouse')

sns.despine(top = True, right = True)

plt.legend(bbox_to_anchor=(1.05, 1),
           loc='upper left', 
           borderaxespad=0, 
           title = 'Reward Volume (uL)', 
           title_fontsize = 6,
           fontsize = 6)

plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/ECDF_LicksInBoutDays2&3_ProjectType.svg', dpi=300, bbox_inches = 'tight')
plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/ECDF_LicksInBoitDays2&3_ProjectType.png', dpi=300, bbox_inches = 'tight')

In [ ]:
plt.figure(figsize = (5,4), dpi=300)
proj_palette = [projcmap[x] for x in trialdf['Mouse'].unique().tolist()]

g = sns.lineplot(data = sessionlickboutdf[(sessionlickboutdf['Day'].isin(['Day4'])) &
                                      (sessionlickboutdf['Session Loom Number'] < 11) &
                                      (sessionlickboutdf['Most Recent Volume'] == 2000) &
                                      ((sessionlickboutdf['Session Loom Number'] == 0) | (sessionlickboutdf['Lick Bout with Loom'])) &
                                      (sessionlickboutdf['Trial Lick Bout Number'].isin([1]))],
            x = 'Session Loom Number',
            y= 'Licks in Bout',
            hue = 'Mouse',
            hue_order = trialdf['Mouse'].unique().tolist(),
            palette = proj_palette,
            ci = None,
            alpha = 0.6)
g.set_xticks([0,1,2,3,4,5,6,7,8,9,10])
g.set_xticklabels(['No\nLoom',1,2,3,4,5,6,7,8,9,10])
g.set(title = 'Day4 (Dine and Dash)\nLicks in First Bout\n of 25µL Trial')

sns.despine(top = True, right = True)

leg = plt.legend(bbox_to_anchor=(1.05, 1),
           loc='upper left', 
           borderaxespad=0, 
           title = 'ZI Population Imaged', 
           labels = ['ZI-SC', 'ZI-PRN'],
           title_fontsize = 8,
           fontsize = 8)

leg.legendHandles[0].set_color('cyan')
leg.legendHandles[1].set_color('magenta')

plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/LinePlot_LicksInFirst25uLBout_vs_LoomNum_ProjectType.svg', dpi=300, bbox_inches = 'tight')
plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/LinePlot_LicksInFirst25uLBout_vs_LoomNum_ProjectType.png', dpi=300, bbox_inches = 'tight')

In [ ]:

for mouse in sessionlickboutdf[sessionlickboutdf['Day'] == 'Day4']['Mouse'].unique().tolist():
    plt.figure(figsize = (5,4), dpi=300)
    proj_palette = [projcmap[x] for x in trialdf['Mouse'].unique().tolist()]

    g = sns.pointplot(data = sessionlickboutdf[ (sessionlickboutdf['Mouse'] == mouse) &
                                           (sessionlickboutdf['Day'].isin(['Day4'])) &
                                          (sessionlickboutdf['Session Loom Number'] < 11) &
                                          (sessionlickboutdf['Most Recent Volume'] >= 1000) &
                                          ((sessionlickboutdf['Session Loom Number'] == 0) | (sessionlickboutdf['Lick Bout with Loom'])) &
                                          (sessionlickboutdf['Trial Lick Bout Number'].isin([1]))],
                x = 'Session Loom Number',
                y= 'Licks in Bout',
                hue = 'Mouse',
                errorbar = 'se',
                palette = [proj2cmap[mouse]])
    
    g.set_xticks([0,1,2,3,4,5,6,7,8,9,10])
    g.set_xticklabels(['No\nLoom',1,2,3,4,5,6,7,8,9,10])
    g.set(title = 'Licks in First Bout\nof 25µL Trial on Day 4\n(Dine and Dash)', ylim = (0,80))

    sns.despine(top = True, right = True)

    leg = plt.legend(bbox_to_anchor=(1.05, 1),
               loc='upper left', 
               borderaxespad=0, 
               title = 'Mouse ID', 
               title_fontsize = 8,
               fontsize = 8)

    #leg.legendHandles[0].set_color('cyan')
    #leg.legendHandles[1].set_color('magenta')

    plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/LinePlot_LicksInFirst25uLBout_vs_LoomNum_' + mouse + '.svg', dpi=300, bbox_inches = 'tight')
    plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/LinePlot_LicksInFirst25uLBout_vs_LoomNum' + mouse + '.png', dpi=300, bbox_inches = 'tight')

In [ ]:
for mouse in sessionlickboutdf['Mouse'].unique().tolist():
    plt.figure(figsize = (5,4), dpi=300)
    proj_palette = [projcmap[x] for x in trialdf['Mouse'].unique().tolist()]

    g = sns.lineplot(data = sessionlickboutdf[ (sessionlickboutdf['Mouse'] == mouse)
                                           (sessionlickboutdf['Day'].isin(['Day4'])) &
                                          (sessionlickboutdf['Session Loom Number'] < 11) &
                                          (sessionlickboutdf['Most Recent Volume'] == 2000) &
                                          ((sessionlickboutdf['Session Loom Number'] == 0) | (sessionlickboutdf['Lick Bout with Loom'])) &
                                          (sessionlickboutdf['Trial Lick Bout Number'].isin([1]))],
                x = 'Session Loom Number',
                y= 'Licks in Bout',
                hue = 'Mouse',
                hue_order = trialdf['Mouse'].unique().tolist(),
                palette = proj_palette,
                ci = None,
                alpha = 0.6)
    g.set_xticks([0,1,2,3,4,5,6,7,8,9,10])
    g.set_xticklabels(['No\nLoom',1,2,3,4,5,6,7,8,9,10])
    g.set(title = 'Licks in First Bout\nof 25µL Trial\non Day 4 (Dine and Dash)')

    sns.despine(top = True, right = True)

    leg = plt.legend(bbox_to_anchor=(1.05, 1),
               loc='upper left', 
               borderaxespad=0, 
               title = 'ZI Population Imaged', 
               labels = ['ZI-SC', 'ZI-PRN'],
               title_fontsize = 8,
               fontsize = 8)

    leg.legendHandles[0].set_color('cyan')
    leg.legendHandles[1].set_color('magenta')

    plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/LinePlot_LicksInFirst25uLBout_vs_LoomNum_' + mouse '.svg', dpi=300, bbox_inches = 'tight')
    plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/LinePlot_LicksInFirst25uLBout_vs_LoomNum' + mouse '.png', dpi=300, bbox_inches = 'tight')

In [ ]:
plt.figure(figsize = (4,3), dpi =300)

DATA_N = sessionlickepochdf
AAA = DATA_N.loc[DATA_N[DATA_N['On Target Lick Bout']].groupby(['Mouse', 'Day', 'Trial Number at Start of Lick Epoch'])['Session Lick Epoch Number'].idxmin()].reset_index(drop=True)

ax = sns.boxplot(data = AAA[(AAA['Day'] == 'Day4') & (AAA['Most Recent Volume'] == 2000) &
                            ((AAA['Session Loom Number'] == 0) | (AAA['Lick Epoch with Loom'])) &
                            (AAA['Session Loom Number'] < 11)].groupby(['Session Loom Number', 'Mouse']).mean().reset_index(),
             x = 'Session Loom Number',
             y = 'Licks in Epoch',
             color = 'gray', showfliers = False)

ax = sns.swarmplot(data = AAA[(AAA['Day'] == 'Day4') & (AAA['Most Recent Volume'] == 2000) &
                              ((AAA['Session Loom Number'] == 0) | (AAA['Lick Epoch with Loom'])) &
                                   (AAA['Session Loom Number'] < 11)].groupby(['Session Loom Number', 'Mouse']).mean().reset_index(),
             x = 'Session Loom Number',
             y = 'Licks in Epoch',
             color = 'darkturquoise',
             edgecolor = 'black',
             linewidth = 0.5,
             size = 5,
             alpha = 0.3)


sns.despine(top = True, right = True)

plt.title('Number of Licks During\nFirst Lick Epoch (Interlick Interval < 3 second)\nof Each Large Volume (25uL) Loom Trial')

plt.legend([],[], frameon=False)

#plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/BoxStrip_LicksInFirst25uLEpoch_vs_LoomNum.svg', dpi=300, bbox_inches = 'tight')
#plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/BoxStrip_LicksInFirst25uLEpoch_vs_LoomNum.png', dpi=300, bbox_inches = 'tight')

In [ ]:
plt.figure(figsize = (8,4), dpi =300)

DATA_N = sessionlickboutdf
AAA = DATA_N.loc[DATA_N[DATA_N['On Target Lick Bout']].groupby(['Mouse', 'Day', 'Trial Number at Start of Lick Bout'])['Session Lick Bout Number'].idxmin()].reset_index(drop=True)

ax = sns.boxplot(data = AAA[(AAA['Day'] == 'Day4') & (AAA['Most Recent Volume'] == 2000) &
                            ((AAA['Session Loom Number'] == 0) | (AAA['Lick Bout with Loom'])) &
                            (AAA['Session Loom Number'] < 11)].groupby(['Session Loom Number', 'Mouse']).mean().reset_index(),
             x = 'Session Loom Number',
             y = 'Licks in Bout',
             color = 'white',
             showfliers=False)

ax = sns.swarmplot(data = AAA[(AAA['Day'] == 'Day4') & (AAA['Most Recent Volume'] == 2000) &
                            ((AAA['Session Loom Number'] == 0) | (AAA['Lick Bout with Loom'])) &
                            (AAA['Session Loom Number'] < 11)].groupby(['Session Loom Number', 'Mouse']).mean().reset_index(),
              x = 'Session Loom Number',
             y = 'Licks in Bout',
             color = 'black',
             edgecolor = 'black',
             linewidth = 1,
             dodge = True,
             size = 5,
             alpha = 0.3)

ax.set_xticklabels(['Baseline','1','2','3','4','5','6','7','8','9','10'])

sns.despine(top = True, right = True)

plt.title('Number of Licks During\nFirst Lick Bout (Interlick Interval < 1 second)\nof Each Large Volume (25uL) Loom Trial')

plt.legend([],[], frameon=False)

#plt.savefig('/Users/ryankast/Documents/FengLab/Manuscripts/ZI_CellTypes/V4/Fig5/LicksInBout_vs_LoomNum.svg', dpi=300, bbox_inches = 'tight')
#plt.savefig('/Users/ryankast/Documents/FengLab/Manuscripts/ZI_CellTypes/V4/Fig5/LicksInBout_vs_LoomNum.png', dpi=300, bbox_inches = 'tight')

In [ ]:
plt.figure(figsize = (6,4), dpi =300)

DATA_N = sessionlickboutdf
AAA = DATA_N.loc[DATA_N[DATA_N['On Target Lick Bout']].groupby(['Mouse', 'Day', 'Trial Number at Start of Lick Bout'])['Session Lick Bout Number'].idxmin()].reset_index(drop=True)



ax = sns.lineplot(data = AAA[(AAA['Day'] == 'Day4') & (AAA['Most Recent Volume'] == 2000) &
                            ((AAA['Session Loom Number'] == 0) | (AAA['Lick Bout with Loom'])) &
                            (AAA['Session Loom Number'] < 11)].groupby(['Session Loom Number', 'Mouse']).mean().reset_index(),
              x = 'Session Loom Number',
             hue = 'Mouse',
             y = 'Licks in Bout',
             color = 'black',
             linewidth = 1,
             size = 5,
             alpha = 0.3)

ax.set_xticklabels(['Baseline','1','2','3','4','5','6','7','8','9','10'])

sns.despine(top = True, right = True)

plt.title('Number of Licks During\nFirst Lick Bout (Interlick Interval < 1 second)\nof Each Large Volume (25uL) Loom Trial')

plt.legend([],[], frameon=False)

#plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/BoxStrip_LicksInFirst25uLEpoch_vs_LoomNum.svg', dpi=300, bbox_inches = 'tight')
#plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/BoxStrip_LicksInFirst25uLEpoch_vs_LoomNum.png', dpi=300, bbox_inches = 'tight')

In [ ]:
dest_dir = 'Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DLC_DineAndDash/'

for (mouse,day,AP), group in dfFullSet[dfFullSet['Day'] == 'Day4'].groupby(['Mouse','Day', 'Active Port']):
    df = pd.DataFrame()
    df['ON timestamps'] = group[group['Event'].str.contains('started')]['Guppy Timestamps (s)'].reset_index(drop = True)
    df['OFF timestamps'] = group[group['Event'].str.contains('is ending')]['Guppy Timestamps (s)'].reset_index(drop = True)
    df.to_csv(dest_dir + mouse + '_' + day + '_LED' + AP + '_Timestamps.csv')



In [ ]:
plt.figure(figsize = (6,4), dpi =300)

DATA_N = sessionlickepochdf
AAA = DATA_N.loc[DATA_N[DATA_N['On Target Lick Bout']].groupby(['Mouse', 'Day', 'Trial Number at Start of Lick Epoch'])['Session Lick Epoch Number'].idxmin()].reset_index(drop=True)

ax = sns.lineplot(data = AAA[(AAA['Day'] == 'Day4') & (AAA['Most Recent Volume'] == 2000) &
                            ((AAA['Session Loom Number'] == 0) | (AAA['Lick Epoch with Loom'])) &
                            (AAA['Session Loom Number'] < 11)].groupby(['Session Loom Number', 'Mouse']).mean().reset_index(),
             x = 'Session Loom Number',
             y = 'Licks in Epoch',\
             hue = 'Projection Type')

ax = sns.stripplot(data = AAA[(AAA['Day'] == 'Day4') & (AAA['Most Recent Volume'] == 2000) &
                              ((AAA['Session Loom Number'] == 0) | (AAA['Lick Epoch with Loom'])) &
                                   (AAA['Session Loom Number'] < 11)].groupby(['Session Loom Number', 'Mouse']).mean().reset_index(),
             x = 'Session Loom Number',
             y = 'Licks in Epoch',
             color = 'darkturquoise',
             edgecolor = 'black',
             linewidth = 0.5,
             size = 5,
             alpha = 0.3)


sns.despine(top = True, right = True)

plt.title('Number of Licks During\nFirst Lick Epoch (Interlick Interval < 3 second)\nof Each Large Volume (25uL) Loom Trial')

plt.legend([],[], frameon=False)

In [ ]:
pd.read_hdf("Z:\RKDATA\Behavior\FiberPhotometryData\Analysis\DND_DiscreteAnalysis\GuppyFormatCsvs12_14_23\Day3\JAX-15F-2L\JAX-15F-2L_output_1\FirstLick-BothPorts_60000ms_ZI-PRN_z_score_ZI-PRN.h5")

In [ ]:
sessionlickboutdf[['Mouse','Day','Session Lick Bout Number']].to_dict()

In [ ]:
dict_list = sessionlickboutdf.to_dict(orient='records')

# Create the dictionary
bout_dict = {}

# Iterate through the list of dictionaries
for item in dict_list:
    # Create a tuple of the specified columns as key
    key = (item['Mouse'], item['Day'], item['Session Lick Bout Number'])
    # Assign 'Licks in Bout' column value as the dictionary value
    value = item['Licks in Bout']
    # Assign key-value pair to the dictionary
    bout_dict[key] = value

In [ ]:
dfMerged.columns.to_list()

In [ ]:
df[(df['Mouse'] == target_mouse) 
                   & (df['Day'] == 'Day4')
                   & (((df['Event'] == 'Lick detected') & (df['On Target Lick'] == True)) | (df['Input'] == 'Input_3'))
                   #& (df['Most Recent Volume'] == 2000)
                   ]


In [ ]:
lick_data

In [ ]:
dfMerged[dfMerged['Day'] == 'Day3']['Most Recent Volume'].value_counts()

In [ ]:
plt.figure(figsize =(4,8) ,dpi=300)

df = dfMerged
# Assuming your DataFrame is named 'df'
# Convert 'Arduino_Timestamps' to datetime format
df['Arduino_Timestamps'] = pd.to_datetime(df['Arduino_Timestamps'], unit='ms')


for mouse in df[(df['Day'] == 'Day4')]['Mouse'].unique().tolist():
     # Create a raster plot for the selected lick bouts
    fig, ax = plt.subplots(figsize=(2,3), dpi = 300)
    # Specify the mouse ID and the number of lick bouts to plot
    target_mouse = mouse

    # Extract relevant columns for the plot
    selected_bouts = df[(df['Mouse'] == target_mouse) 
                   & (df['Day'] == 'Day4')
                   & (df['Active Port'] == 4)
                       & (((df['Event'] == 'Lick detected')   
                           & (df['On Target Lick'] == True) 
                           & (df['Most Recent Volume'] >= 1500)) 
                              | (df['Input'] == 'Input_3'))]

    # Iterate over each trial
    selectedlist = []
    boutnum = 0
    
    for bout, bout_df in selected_bouts.groupby('Trial Number'):
        boutnum += 1
        for i in range(len(bout_df)):
            bout_df['Time Elapsed'] = (bout_df['Arduino_Timestamps'] - bout_df[bout_df['Event'] == 'Lick detected']['Arduino_Timestamps'].min()).dt.total_seconds()
            bout_df['Bout'] = boutnum
        selectedlist.append(bout_df)

    selected_bouts = pd.concat(selectedlist)

    # Plot each lick at the corresponding time elapsed with a fixed y level for the trail
    g = sns.scatterplot(data = selected_bouts[selected_bouts['Input'] != 'Input_3'],
                    x = 'Time Elapsed', 
                    y = 'Bout',
                    style = 'Input',
                    color = 'darkgrey',
                    s = 4,
                    zorder = 2)
                                              
    g = sns.scatterplot(data = selected_bouts[selected_bouts['Input'] == 'Input_3'],
                    x = 'Time Elapsed', 
                    y = 'Bout', 
                    s = 4,
                    style = 'Input',
                    color = 'red',
                    alpha = 0.5,
                    zorder = 3)
    
    handles, labels = g.get_legend_handles_labels()
    # Customize the plot

    sns.despine(top = True, right = True)
    ax.set_xlabel('Time From First Lick (s)')
    ax.set_ylabel('Trial Number')
    ax.set_xlim(-0.25,6)
    plt.title(mouse, pad = 20)
    plt.legend(bbox_to_anchor=(1.1, 1),
               loc='upper left',
               title = 'Event',
               labels = ['Lick', 'Loom'],
               handles = handles,
               borderaxespad=0, 
               title_fontsize = 8,
               fontsize = 8, markerscale = 1)
    
    for i in selected_bouts[selected_bouts['Input'] == 'Input_3']['Bout'].unique().tolist():
        plt.axhline(y=i, color ='red', linewidth = 0.1, zorder =1)

    #plt.savefig('/Volumes/ryanKast/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/MousePlots/' + mouse + '_Port425uL_LickLoomRaster.svg', dpi=300, bbox_inches = 'tight')
    #plt.savefig('/Volumes/ryanKast/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/MousePlots/' + mouse + '_Port425uL_LickLoomRaster.png', dpi=300, bbox_inches = 'tight')


In [ ]:
bout_data

In [ ]:
boutlicks = []
for i in range(len(dfFullSet)):
    if dfFullSet.iloc[i]['Event'] == 'Lick detected':
        mouse = dfFullSet.iloc[i]['Mouse']
        day = dfFullSet.iloc[i]['Day']
        lb = dfFullSet.iloc[i]['Session Lick Bout Number']
        
        boutlicks.append(sessionlickboutdf[(sessionlickboutdf['Mouse'] == mouse) &
                          (sessionlickboutdf['Day'] == day) &
                          (sessionlickboutdf['Session Lick Bout Number'] == lb)]['Licks in Bout'].iloc[0])
    else:
        boutlicks.append(np.nan)
    

In [ ]:
dfFullSet['Licks in Bout'] = boutlicks

In [ ]:
dfFullSet

In [ ]:
g = sns.FacetGrid(sessionlickboutdf[(sessionlickboutdf['Day'].isin(['Day4'])) &
                                      (sessionlickboutdf['Session Loom Number'] < 11) &
                                      (sessionlickboutdf['Most Recent Volume'] == 2000) &
                                      ((sessionlickboutdf['Session Loom Number'] == 0) | (sessionlickboutdf['Lick Bout with Loom'])) &
                                      (sessionlickboutdf['Trial Lick Bout Number'].isin([1]))], col="Projection Type", row="Mouse", margin_titles=True, height=3, aspect=2)

# Plot a line for each combination of 'Mouse' and 'ProjectionType'
g.map(sns.lineplot, "Session Loom Number", "Licks in Bout")

# Add a title to each facet
g.set_titles(col_template="{col_name}", row_template="{row_name}")

# Adjust the layout
g.tight_layout()

# Show the plot
plt.show()

In [ ]:
plt.figure(figsize = (6,4), dpi =300)

DATA_N = sessionlickboutdf
AAA = DATA_N.loc[DATA_N[DATA_N['On Target Lick Bout']].groupby(['Mouse', 'Day', 'Trial Number at Start of Lick Bout'])['Session Lick Bout Number'].idxmin()].reset_index(drop=True)

ax = sns.boxplot(data = AAA[(AAA['Most Recent Volume'] == 2000) & (AAA['Day'] == 'Day4') & 
                            ((AAA['Session Loom Number'] == 0) | (AAA['Lick Bout with Loom'])) &
                            (AAA['Session Loom Number'] < 11)].groupby(['Session Loom Number', 'Projection Type', 'Mouse']).mean().reset_index(),
             x = 'Session Loom Number',
             y = 'Licks in Bout',
             hue = 'Projection Type',
             palette = ['Magenta', 'Cyan'])

ax = sns.swarmplot(data = AAA[(AAA['Most Recent Volume'] == 2000) & (AAA['Day'] == 'Day4') &
                              ((AAA['Session Loom Number'] == 0) | (AAA['Lick Bout with Loom'])) &
                                   (AAA['Session Loom Number'] < 11)].groupby(['Session Loom Number', 'Projection Type', 'Mouse']).mean().reset_index(),
             x = 'Session Loom Number',
             y = 'Licks in Bout',
             hue = 'Projection Type',
             palette = ['Magenta', 'Cyan'],
             linewidth = 0.8,
             size = 5,
             alpha = 0.6,
             dodge = True)


sns.despine(top = True, right = True)

plt.title('Number of Licks During\nFirst Lick Bout(Interlick Interval < 1 second)\nof Each Large Volume (25uL) Loom Trial')

plt.legend([],[], frameon=False)

In [ ]:
sns.lineplot(data = sessionlickepochdf[(sessionlickepochdf['Lick Epoch with Loom']) | 
                                      (sessionlickepochdf['Day'] == 'Day4')].groupby(['Mouse','Session Loom Number']).reset_index(),
             x= 'Session Loom Number',
             y= 'Licks in Epoch',
             hue = 'Mouse',
             ci = None
           )


plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Mouse ID', 
           title_fontsize = 8,
           fontsize = 8)

In [ ]:
tlbn_list = []
for i in range(len(sessionlickboutdf)):
    mouse = sessionlickboutdf['Mouse'].iloc[i]
    day = sessionlickboutdf['Day'].iloc[i]
    slbn = sessionlickboutdf['Session Lick Bout Number'].iloc[i]
               
    tlbn = dfFullSet[(dfFullSet['Mouse'] == mouse) &
              (dfFullSet['Day'] == day) &
              (dfFullSet['Session Lick Bout Number'] == slbn)].iloc[0]['Trial Lick Bout Number']
               
    tlbn_list.append(tlbn)          

In [ ]:
file = "Z:\RKDATA\Behavior\FiberPhotometryData\Analysis\DLC_DineAndDash\Remainder\JAX-15F-2L_Arena_1DLC_resnet50_FOL_DLC_V3Aug8shuffle1_70000.h5"
pd.read_hdf(file).to_csv(file.replace(".h5", ".csv"))

In [ ]:
plt.figure(figsize = (4,3), dpi = 300)
sns.lineplot(data = sessionlickboutdf[((sessionlickboutdf['Most Recent Volume'] == 2000) | sessionlickboutdf['Session Loom Number'] > 0 ) & 
                                      (sessionlickboutdf['Day'] == 'Day4') &
                                      (sessionlickboutdf['Trial Lick Bout Number'] == 1) &
                                      (sessionlickboutdf['Session Loom Number'] < 11) &
                                      ((sessionlickboutdf['Lick Bout with Loom'])
                                       | (sessionlickboutdf['Session Loom Number'] == 0)
                                      )].groupby(['Mouse','Session Loom Number']).mean().reset_index(),
             x= 'Session Loom Number',
             y= 'Licks in Bout after Loom Onset',
             hue = 'Mouse',
           )


plt.legend(bbox_to_anchor=(1.1, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Mouse ID', 
           title_fontsize = 6,
           fontsize = 6)

In [ ]:
plt.figure(figsize = (4,3), dpi = 300)
sns.swarmplot(data = sessionlickboutdf[((sessionlickboutdf['Most Recent Volume'] == 2000)
                                        | sessionlickboutdf['Session Loom Number'] > 0 ) & 
                                      (sessionlickboutdf['Day'] == 'Day4') &
                                      (sessionlickboutdf['Trial Lick Bout Number'] == 1) &
                                      (sessionlickboutdf['Session Loom Number'] < 11) &
                                      ((sessionlickboutdf['Lick Bout with Loom'])
                                       | (sessionlickboutdf['Session Loom Number'] == 0)
                                      )].groupby(['Mouse','Session Loom Number']).mean().reset_index(),
             x= 'Session Loom Number',
             y= 'Licks in Bout',
             hue = 'Mouse',
             alpha = 0.4
           ).set(ylim = (0,20))


plt.legend(bbox_to_anchor=(1.1, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Mouse ID', 
           title_fontsize = 6,
           fontsize = 6)

In [ ]:
sessionlickboutdf[(sessionlickboutdf['Licks in Bout'] == 49) &
(sessionlickboutdf['Session Loom Number'] ==1)]
                 # (sessionlickboutdf['Lick Bout with Loom'])]['Licks in Bout'].max()

In [ ]:
plt.figure(figsize = (4,3), dpi = 300)
ax = sns.lineplot(data = sessionlickboutdf[((sessionlickboutdf['Most Recent Volume'] == 2000)
                                        | sessionlickboutdf['Session Loom Number'] > 0 ) & 
                                      (sessionlickboutdf['Day'] == 'Day4') &
                                      (sessionlickboutdf['Trial Lick Bout Number'] < 2) &
                                      (sessionlickboutdf['Session Loom Number'] < 11) &
                                      ((sessionlickboutdf['Lick Bout with Loom'])
                                       | (sessionlickboutdf['Session Loom Number'] < 2)
                                      )].groupby(['Mouse','Session Loom Number']).mean().reset_index(),
             x= 'Session Loom Number',
             y= 'Licks in Bout after Loom Onset',
             hue = 'Mouse',
             alpha =0.5)

ax = sns.swarmplot(data = sessionlickboutdf[((sessionlickboutdf['Most Recent Volume'] == 2000)
                                        | sessionlickboutdf['Session Loom Number'] > 0 ) & 
                                      (sessionlickboutdf['Day'] == 'Day4') &
                                      (sessionlickboutdf['Trial Lick Bout Number'] == 1) &
                                      (sessionlickboutdf['Session Loom Number'] < 11) &
                                      ((sessionlickboutdf['Lick Bout with Loom']) #Might need to remove this filter
                                       | (sessionlickboutdf['Session Loom Number'] < 2)
                                      )].groupby(['Mouse','Session Loom Number']).mean().reset_index(),
             x= 'Session Loom Number',
             y= 'Licks in Bout after Loom Onset',
             hue = 'Mouse',
            )

plt.legend(bbox_to_anchor=(1.1, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Mouse ID', 
           title_fontsize = 6,
           fontsize = 6)

In [ ]:
plt.figure(figsize = (6,3), dpi =300)


sns.lineplot(data= session_successrate,
              x = 'Day',
              y = 'Success Rate',
              hue = 'Mouse',
              palette = 'tab10')


plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Mouse ID', 
           title_fontsize = 8,
           fontsize = 8)

In [ ]:
plt.figure(figsize = (3,2), dpi =300)
sns.scatterplot(data= sessionlickboutdf[sessionlickboutdf['Licks in Bout'] > 2],
                x = 'Licks in Bout',
                y = 'Mean Bout Frequency (Hz)', 
                #hue = 'Active Port',
                palette = ['darkturquoise','red'],
                alpha =0.05, color = 'gray', s = 4)

sns.despine(top=True, right = True)

plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Active Port', 
           title_fontsize = 8,
           fontsize = 8)

#plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/Scatter_MeanFreq_vs_LickNumber.svg', dpi=300, bbox_inches = 'tight')
#plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/Scatter_MaxFreq_vs_LickNumber.png', dpi=300, bbox_inches = 'tight')

In [ ]:
sessionlickboutdf[sessionlickboutdf['Licks in Bout'] > 2]

In [ ]:
sns.swarmplot(data = SessionResponseTimes[SessionResponseTimes['Trials Since Loom'] < 0],
              x = 'Day',
              y = 'Successful Response Time',
              hue = 'Active Port')

plt.legend([],[], frameon=False)

In [ ]:
DATA_AA = sessionlickboutdf.groupby(['Mouse', 'Day','Trial Number at Start of Lick Bout']).max().reset_index()

sns.lineplot(data = DATA_AA[(DATA_AA['Active Port'] == '4')],
            x = (DATA_AA['Most Recent Volume'] / 100) + 5,
            y = 'Licks in Bout',
            hue = 'Projection Type').set(ylabel = 'Licks in Longest Bout', xlabel = 'Most Recent Volume (uL)')

plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Projection Type Labeled', 
           title_fontsize = 8,
           fontsize = 8,
           )

In [ ]:
g = sns.FacetGrid(sessionlickboutdf[sessionlickboutdf['Lick Bout with Loom']],
                  row='Mouse', col = 'Projection Type', height=4)

# Map the line plot onto the grid
g.map(sns.lineplot, 'Session Loom Number', 'Licks in Bout').set(xlim=(-1,10))

# Add legend
g.add_legend(title='Mouse ID', title_fontsize=8, fontsize=8, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)


In [ ]:
sns.lineplot(data = sessionlickboutdf[sessionlickboutdf['Lick Bout with Loom']],
             y = 'Licks in Bout after Loom Onset',
             x = 'Session Loom Number',
            ).set_xlim(-1,)

plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Mouse ID', 
           title_fontsize = 8,
           fontsize = 8)

In [ ]:
sessionlickepochdf

In [ ]:
trialdf

In [ ]:
trialdf[(trialdf['Volumes'] == 2000) &
                             (trialdf['Day'] == 'Day4') &
                             (trialdf['Session Loom Number'] < 11) &
                             (trialdf['Active Port'] == '4')].groupby(['Mouse', 'Projection Type', 'Day', 'Session Loom Number']).mean()

In [ ]:
plt.figure(figsize = (5,3), dpi=300)

ax = sns.swarmplot(data = trialdf[(trialdf['Volumes'] == 2000) &
                             (trialdf['Day'] == 'Day4') &
                             (trialdf['Session Loom Number'] < 11) &
                             (trialdf['Active Port'] == '4')].groupby(['Mouse', 'Projection Type', 'Day', 'Session Loom Number']).mean().reset_index(),
                x = 'Session Loom Number',
                y = 'On Target Lick Count',
                alpha = 0.6,
                hue = 'Projection Type',
                palette = ['Magenta', 'Cyan'])

ax = sns.boxplot(data = trialdf[(trialdf['Volumes'] == 2000) &
                             (trialdf['Day'] == 'Day4') &
                             (trialdf['Session Loom Number'] < 11) &
                             (trialdf['Active Port'] == '4')].groupby(['Mouse', 'Projection Type', 'Day', 'Session Loom Number']).mean().reset_index(),
                x = 'Session Loom Number',
                y = 'On Target Lick Count',
                color = 'lightgrey')
                #hue = 'Projection Type',
                #palette = ['Magenta', 'Cyan'])

sns.despine(top = True, right = True)

plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Projection Type Labeled', 
           title_fontsize = 8,
           fontsize = 8)

plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/BoxSwarm_OnTargetLickCount_vs_SessionLoomNumberTo10.svg', dpi=300, bbox_inches = 'tight')
plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/BoxSwarm_OnTargetLickCount_vs_SessionLoomNumberTo10.png', dpi=300, bbox_inches = 'tight')

In [ ]:
plt.figure(figsize = (5,3), dpi=300)

proj_palette = [projcmap[x] for x in trialdf['Mouse'].unique().tolist()]
ax = sns.lineplot(data = trialdf[(trialdf['Volumes'] == 2000) &
                             (trialdf['Day'] == 'Day4') &
                             (trialdf['Session Loom Number'] < 11) &
                             (trialdf['Active Port'] == '4')].groupby(['Mouse', 'Projection Type', 'Day', 'Session Loom Number']).mean().reset_index(),
                x = 'Session Loom Number',
                y = 'On Target Lick Count',
                alpha = 0.6,
                hue = 'Mouse',
                hue_order = trialdf['Mouse'].unique().tolist(),
                palette = proj_palette,
                ci = None)


sns.despine(top = True, right = True)


plt.legend([],[], frameon=False)

plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/Proj_Lineplot_OnTargetLickCount_vs_SessionLoomNumberTo10.svg', dpi=300, bbox_inches = 'tight')
plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/Proj_Lineplot_OnTargetLickCount_vs_SessionLoomNumberTo10.png', dpi=300, bbox_inches = 'tight')

In [ ]:
plt.figure(figsize = (5,3), dpi=300)

proj_palette = [projcmap[x] for x in trialdf['Mouse'].unique().tolist()]
ax = sns.lineplot(data = trialdf[(trialdf['Volumes'] == 2000) &
                             (trialdf['Day'] == 'Day4') &
                             (trialdf['Session Loom Number'] < 11) &
                             (trialdf['Active Port'] == '4')].groupby(['Mouse', 'Projection Type', 'Day', 'Session Loom Number']).mean().reset_index(),
                x = 'Session Loom Number',
                y = 'On Target Lick Count',
                alpha = 0.6,
                hue = 'Mouse',
                hue_order = trialdf['Mouse'].unique().tolist(),
                palette = proj_palette,
                ci = None)


sns.despine(top = True, right = True)


plt.legend([],[], frameon=False)

plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/Proj_Lineplot_OnTargetLickCount_vs_SessionLoomNumberTo10.svg', dpi=300, bbox_inches = 'tight')
plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/Proj_Lineplot_OnTargetLickCount_vs_SessionLoomNumberTo10.png', dpi=300, bbox_inches = 'tight')

In [ ]:
ct_cmap = [ctcmdict[x] for x in celltypes]

sns.lineplot(data= session_successrate,
              x = 'Day',
              y = 'Success Rate',
              hue = 'Cell Type',
              hue_order = celltypes,
              palette = ct_cmap)

In [ ]:
plt.figure(figsize = (6,3), dpi=200)

ax = sns.swarmplot(data = trialdf[
                             (trialdf['Day'] == 'Day4') &
                             (trialdf['Session Loom Number'] < 11) &
                             (trialdf['Active Port'] == '4')].groupby(['Mouse', 'Projection Type', 'Day', 'Volumes', 'Trials Since Loom']).mean().reset_index(),
                x = 'Trials Since Loom',
                y = 'On Target Lick Count',
                alpha = 0.6,
                hue = 'Volumes',)
                #palette = ['Magenta', 'Cyan'])

ax = sns.boxplot(data = trialdf[
                             (trialdf['Day'] == 'Day4') &
                             (trialdf['Session Loom Number'] < 11) &
                             (trialdf['Active Port'] == '4')].groupby(['Mouse', 'Projection Type', 'Day', 'Volumes', 'Trials Since Loom']).mean().reset_index(),
                x = 'Trials Since Loom',
                y = 'On Target Lick Count',
                color = 'lightgrey')
                #hue = 'Projection Type',
                #palette = ['Magenta', 'Cyan'])

sns.despine(top = True, right = True)

plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Projection Type Labeled', 
           title_fontsize = 8,
           fontsize = 8)

In [ ]:
trialdf

In [ ]:
plt.figure(figsize=(4,3),dpi=300)
sns.lineplot(data = trialdf[ (~trialdf['Premature Lick']) &
                             (trialdf['Session Loom Number'] < 1) &
                             (trialdf['On Target Lick Count'] >= 3) &
                             (trialdf['Day'] != 'Day1') &
                             (trialdf['Loom Trial'] == False)],
                x = (trialdf['Volumes'] / 100)+ 5,
                y = 'On Target Lick Count',
                hue = 'Active Port',
                palette = ['darkturquoise', 'red']).set(xlabel = 'Reward Volume\n(uL Milk)')


sns.despine(right = True, top = True )
plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Active Port', 
           title_fontsize = 8,
           fontsize = 8)

plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/Lineplot_OnTargetLickCount_vs_RewardVolume.svg', dpi=300, bbox_inches = 'tight')
plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/Lineplot_OnTargetLickCount_vs_RewardVolume.png', dpi=300, bbox_inches = 'tight')

In [ ]:
plt.figure(figsize=(4,3),dpi=300)

sns.boxplot(data = trialdf[  (~trialdf['Premature Lick']) &
                             (trialdf['Session Loom Number'] < 1) &
                             (trialdf['On Target Lick Count'] >= 2) &
                             (trialdf['Day'] != 'Day1') &
                             (trialdf['Loom Trial'] == False)],
            
                x = (trialdf['Volumes'] + 500) / 100,
                y = 'On Target Lick Count',
                hue = 'Active Port',
                palette = ['darkturquoise', 'red']).set(ylim = (0,150), xlabel = 'Reward Volume\n(uL Milk)')


sns.despine(right = True, top = True )
plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Active Port', 
           title_fontsize = 8,
           fontsize = 8)

plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/Boxplot_OnTargetLickCount_vs_RewardVolume.svg', dpi=300, bbox_inches = 'tight')
plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/Boxplot_OnTargetLickCount_vs_RewardVolume.png', dpi=300, bbox_inches = 'tight')

In [ ]:
plt.figure(figsize=(4,3),dpi=300)

sns.boxplot(data = trialdf[  (~trialdf['Premature Lick']) &
                             (trialdf['Session Loom Number'] < 1) &
                             (trialdf['On Target Lick Count'] >= 2) &
                             (trialdf['Day'] != 'Day1') &
                             (trialdf['Loom Trial'] == False) &
                             (trialdf['Active Port'] == '4')],
            
                x = (trialdf['Volumes'] + 500) / 100,
                y = 'On Target Lick Count',
                hue = 'Active Port',
                palette = ['red']).set(ylim = (0,150), xlabel = 'Reward Volume\n(uL Milk)')


sns.despine(right = True, top = True )
plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Active Port', 
           title_fontsize = 8,
           fontsize = 8)

plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/Boxplot_Port4only_OnTargetLickCount_vs_RewardVolume.svg', dpi=300, bbox_inches = 'tight')
plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/Boxplot_Port4only_OnTargetLickCount_vs_RewardVolume.png', dpi=300, bbox_inches = 'tight')

In [ ]:
sessionlickboutdf.columns

In [ ]:
sns.swarmplot(data = sessionlickboutdf[(sessionlickboutdf['Licks in Bout'] >= 3) &
                             (sessionlickboutdf['Day'].isin(['Day4'])) &
                             (sessionlickboutdf['Trial Lick Bout Number'].isin([1]))],
                x = 'Most Recent Volume',
                y = 'Lick Bout Duration (ms)',
                hue = 'Lick Bout with Loom',
                palette = ['Magenta', 'Cyan'])

In [ ]:
plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Projection Type Labeled', 
           title_fontsize = 8,
           fontsize = 8)

In [ ]:
sns.boxplot(data = sessionlickboutdf[(sessionlickboutdf['Session Loom Number'] < 1) &
                             (sessionlickboutdf['Day'] != 'Day1')],
                x = 'Most Recent Volume',
                y = 'Lick Bout Duration (ms)',
               # alpha = 1,
                hue = 'Active Port')
                #palette = ['Magenta', 'Cyan'])



plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Projection Type Labeled', 
           title_fontsize = 8,
           fontsize = 8)

In [ ]:
sns.lineplot(data = trialdf[(trialdf['Session Loom Number'] < 1) &
                             (trialdf['On Target Lick Count'] >= 1) &
                             (trialdf['Day'] != 'Day1')],
                x = 'Volumes',
                y = 'On Target Lick Count',
                alpha = 1,
                hue = 'Mouse',
                ci = None)
                #palette = ['Magenta', 'Cyan'])



plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Projection Type Labeled', 
           title_fontsize = 8,
           fontsize = 8)

In [ ]:
plt.figure(figsize = (8,4), dpi =300)
ax = sns.swarmplot(data = trialdf[(trialdf['Volumes'] == 2000) & 
                             (trialdf['Day'] == 'Day4')],
                x = 'Session Loom Number',
                y = 'On Target Lick Count',
                alpha = 0.4,
                hue = 'Projection Type',
                palette = ['Magenta', 'Cyan'],
                dodge = True
                ).set_xlim(-1,)

ax = sns.boxplot(data = trialdf[(trialdf['Volumes'] == 2000) & 
                     
                                (trialdf['Day'] == 'Day4')],
                x = 'Session Loom Number',
                y = 'On Target Lick Count',
                color = 'grey',
                
                ).set_xlim(-1,)

sns.despine(top=True,right = True)

plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Projection Type Labeled', 
           title_fontsize = 8,
           fontsize = 8,
           )

In [ ]:
plt.figure(figsize = (8,4), dpi =300)
sns.swarmplot(data = trialdf[(trialdf['Volumes'] == 2000) & 
                             (trialdf['Day'] == 'Day4')],
                x = 'Session Loom Number',
                y = 'On Target Lick Count',
                alpha = 0.8,
                
                hue = 'Mouse',
                palette = 'tab20',
                ).set_ylim(-5,)

plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Mouse ID', 
           title_fontsize = 8,
           fontsize = 8)

In [ ]:
sns.scatterplot(data = sessionlickboutdf,
               x= 'Licks in Bout',
               y = 'Mean Bout Frequency (Hz)',
               hue = 'Active Port',
               palette = ''
               alpha = 0.5)

In [ ]:
sns.lineplot(data = sessionlickboutdf,
               x= 'Licks in Bout',
               y = 'Lick Bout Duration (ms)',
               hue = 'Active Port')

In [ ]:
plt.figure(figsize = (5,4), dpi = 300)

sns.scatterplot(data = sessionlickboutdf,
               x= 'Licks in Bout',
               y = sessionlickboutdf['Lick Bout Duration (ms)'] / 1000,
               hue = 'Active Port',
               alpha = 0.2,
               s = 8,
               palette = ['darkturquoise', 'red']).set(ylabel = 'Lick Bout Duration (s)')

sns.despine(right = True,
            top = True)

plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Active Port', 
           title_fontsize = 8,
           fontsize = 8)

#plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/Duration_vs_LickNumber.svg', dpi=300, bbox_inches = 'tight')
#plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/Duration_vs_LickNumber.png', dpi=300, bbox_inches = 'tight')

In [ ]:
sessionlickepochdf

In [ ]:
plt.figure(figsize = (5,4), dpi = 300)

sns.swarmplot(data = sessionlickepochdf,
               x= 'Lick Bouts in Epoch',
               y = 'Lick Epoch Duration (ms)',
               hue ='Lick Epoch with Loom',
                alpha=0.3,
                dodge = True,
                s=8,
               palette = ['darkturquoise', 'red'])

In [ ]:
sessionlickepochdf

In [ ]:
sns.histplot(sessionlickepochdf[sessionlickepochdf['Day']=='Day4'],
            x= sessionlickepochdf['Lick Bouts in Epoch'],
            hue = 'Active Port')

In [ ]:
trialdf

In [ ]:
sns.lineplot(trialdf['',
             x= 'Session Loom Number',
             y = 'On Target Lick Count',
             hue = 'Mouse')

In [ ]:
sns.histplot(sessionlickepochdf,
            x = sessionlickepochdf)

In [ ]:
sessionlickboutdf.columns.tolist()


In [ ]:
plt.figure(figsize = (6,3), dpi =300)

celltypes = ['Slc30a3',
             'Nos1',
             'ZI-SC',
             'ZI-PRN']


ct_cmap = [ctcmdict[x] for x in celltypes]

sns.lineplot(data= session_successrate,
              x = 'Day',
              y = 'Success Rate',
              hue = 'Cell Type',
              hue_order = celltypes,
              palette = ct_cmap)


plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Mouse ID', 
           title_fontsize = 8,
           fontsize = 8)

In [ ]:
dfFullSet.columns.tolist()

In [ ]:
sns.barplot(data = dfFullSet[(dfFullSet['Day'] == 'Day4')
                            & (dfFullSet[''])]
            
            

In [ ]:
plt.figure(figsize = (6,3), dpi =300)


sns.lineplot(data= session_successrate,
              x = 'Day',
              y = 'Success Rate',
              #hue = 'Mouse',
              color = 'green')



plt.ylim(0,1.1)

In [ ]:
trialdf[trialdf['Mouse'] == 'JAX-11F-2R']

In [ ]:
DATA_A.mean()

In [ ]:
plt.figure(figsize = (6,3), dpi =300)
sns.lineplot(data= SessionResponseTimes,
              x = 'Day',
              y = SessionResponseTimes['Successful Response Time'] / 1000,
              hue = 'Mouse',
              palette = 'tab10')


plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Mouse ID', 
           title_fontsize = 8,
           fontsize = 8)

plt.ylabel('Successul Response Time (s)')

In [ ]:
dfFullSet[dfFullSet['Trial Lick Bout Number']!= 'NaN']['Event'].value_counts()

In [ ]:
plt.figure(figsize = (6,3), dpi =300)

celltypes = ['Slc30a3',
             'Nos1',
             'ZI-SC',
             'ZI-PRN']


ct_cmap = [ctcmdict[x] for x in celltypes]

sns.lineplot(data= SessionResponseTimes,#[SessionResponseTimes['Cell Type'] == 'Slc30a3'],
              x = 'Day',
              y = SessionResponseTimes['Successful Response Time'] / 1000,
              hue = 'Cell Type',
              hue_order = celltypes,
              palette = ct_cmap)


plt.legend(
           bbox_to_anchor=(1.05, 1),
           loc='upper left', 
           borderaxespad=0, 
           title = 'Cell Type', 
           title_fontsize = 8,
           fontsize = 8)

plt.ylabel('Successul Response Time (s)')

In [ ]:
plt.figure(figsize = (6,3), dpi =300)

celltypes = ['Slc30a3',
             'Nos1',
             'ZI-SC',
             'ZI-PRN']


ct_cmap = [ctcmdict[x] for x in celltypes]

sns.lineplot(data= SessionResponseTimes,#[SessionResponseTimes['Cell Type'] == 'Slc30a3'],
              x = 'Day',
              y = SessionResponseTimes['Successful Response Time'] / 1000,
              hue = 'Cell Type',
              hue_order = celltypes,
              palette = ct_cmap)


plt.legend(
           bbox_to_anchor=(1.05, 1),
           loc='upper left', 
           borderaxespad=0, 
           title = 'Cell Type', 
           title_fontsize = 8,
           fontsize = 8)

plt.ylabel('Successul Response Time (s)')

In [ ]:
plt.figure(figsize = (6,3), dpi =300)
sns.lineplot(data= SessionResponseTimes[SessionResponseTimes['Day'] != 'Day1'],
              x = 'Day',
              y = SessionResponseTimes['Successful Response Time'] / 1000,
              hue = 'Mouse',
              palette = 'tab10')


plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Mouse ID', 
           title_fontsize = 8,
           fontsize = 8)

plt.ylabel('Successul Response Time (s)')

In [ ]:
sns.set_theme(style = 'white', 
              font = 'Helvetica',
              font_scale = 1)
plt.figure(figsize = (6,3), dpi =300)
sns.lineplot(data= SessionResponseTimes[(SessionResponseTimes['Day'] != 'FOL')],
              x = 'Day',
              y = SessionResponseTimes['Successful Response Time'] / 1000,
              hue ='Mouse',
             
              hue_order = mice,
              palette = mouse_cmap,
             legend = False)

plt.ylabel('Average Trial\n On Target Response Time (s)')
plt.xlabel('Session')
sns.despine(top = True, right = True)

#plt.savefig('/Users/ryankast/Documents/FengLab/Meetings/LabMeetings/FLM14/ResponseTime_OverDays.png', dpi = 150, bbox_inches = 'tight')

In [ ]:
sns.set_theme(style = 'white', 
              font = 'arial',
              font_scale = 1)
plt.figure(figsize = (6,3), dpi =300)
sns.swarmplot(data= SessionResponseTimes[(SessionResponseTimes['Active Port'] != np.nan)],
              x = 'Day',
              y = SessionResponseTimes['Successful Response Time'] / 1000,
              hue ='Active Port',
              palette = 'viridis')

plt.ylabel('Average Trial\n On Target Response Time (s)')
plt.xlabel('Session')
sns.despine(top = True, right = True)

plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Active Port', 
           title_fontsize = 8,
           fontsize = 8)

#plt.savefig('/Users/ryankast/Documents/FengLab/Meetings/LabMeetings/FLM14/PortDependent_ResponseTime_OverDays.svg', bbox_inches = 'tight')

In [ ]:
session_successrate[session_successrate['Day'] == 'Day4']

In [ ]:
trialdf

In [ ]:
sns.pointplot(data = trialdf[trialdf['Trial Number'] < 50], x = 'Trial Number', y = 'Successful Response Time', hue = 'Day')

In [ ]:
trialdf

In [ ]:
day4_grouped = trialdf[(trialdf['Day'] == 'Day4') & (trialdf['Trials Since Loom'] >=0) & (trialdf['Trials Since Loom'] <= 10)].groupby(['Mouse', 'Trials Since Loom'])
trial_counts = day4_grouped.size().reset_index(name='Trial Count')

day4_grouped = trialdf[(trialdf['Day'] == 'Day4') & (trialdf['Trials Since Loom'] >=0) & (trialdf['Trials Since Loom'] <= 10)].groupby(['Trials Since Loom'])
mouse_counts = day4_grouped['Mouse'].nunique()

In [ ]:
mouse_counts.index

In [ ]:
plt.figure(figsize = (6,3), dpi = 300)
ax = sns.violinplot(data = trial_counts, x = 'Trials Since Loom', y='Trial Count', color ='white', alpha = 0.5)
sns.scatterplot(data = trial_counts, x = 'Trials Since Loom', y='Trial Count', hue = 'Mouse', palette = 'tab20', s = 6, alpha =0.5)
ax2 = ax.twinx()
sns.lineplot(data = mouse_counts, ax = ax2)
sns.despine(top = True, right = True)

plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           frameon = False,fontsize = 6)

In [ ]:
sns.lineplot(data = mouse_counts)

In [ ]:
trialdf[trialdf['Trials Since Loom'] == 4]['Mouse'].value_counts()

In [ ]:
plt.figure(figsize = (6,4), dpi = 300)

# Get the 'cool_r' colormap
cmap = plt.get_cmap('cool')

# Create a custom colormap that maps negative values to black and uses 'cool_r' for positive values
colors = [(0.5, 0.5, 0.5)] + [cmap(i) for i in np.linspace(0, 1, 256)]

custom_cmap = plt.cm.colors.ListedColormap(colors)


ax = sns.boxplot(data=trialdf[(trialdf['Day'] == 'Day4') &
                                          (trialdf['Trials Since Loom'] < 5)].groupby(['Mouse', 'Trials Since Loom']).mean().reset_index(),
              x = 'Trials Since Loom',
              y = 'Digitized Trial Outcome',
              color = 'white', showfliers = False)



ax = sns.swarmplot(data=trialdf[(trialdf['Day'] == 'Day4') &
                                          (trialdf['Trials Since Loom'] < 5)].groupby(['Mouse', 'Trials Since Loom']).mean().reset_index(),
                x = 'Trials Since Loom',  
                  y = 'Digitized Trial Outcome',
                  hue = 'Mouse',
                  palette = 'tab20')

trialdf2 = trialdf.copy()
trialdf2['Trials Since Loom'] = trialdf['Trials Since Loom'] + 1

ax = sns.lineplot(data=trialdf2[(trialdf2['Day'] == 'Day4') &
                                          (trialdf2['Trials Since Loom'] < 6)].groupby(['Mouse', 'Trials Since Loom']).mean().reset_index(),
                 x = 'Trials Since Loom',  
                  y = 'Digitized Trial Outcome',
                  hue = 'Mouse',
                  palette = 'tab20')

sns.despine(top = True, right = True)

plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
         frameon = False)

ax.set_ylabel('Success Rate (%)')

#plt.savefig('Z:/RKDATA/Behavior/FiberPhotometryData/Analysis/DND_DiscreteAnalysis/BehaviorFigures/SuccessRate_TrialsPastLoom_HueMouse.svg',bbox_inches = 'tight')

In [ ]:
plt.figure(figsize = (6,3), dpi = 300)
plt.ylim(-0.1,1.1)


ax = sns.violinplot(data=session_successrate[(session_successrate['Day'] == 'FOL') &
                                          (session_successrate['Trials Since Loom'] < 6)],
              x = 'Trials Since Loom',
              y = 'Success Rate',
              color = 'lightgrey' ,
              cut = 0,
              legend = False)

ax = sns.swarmplot(data=session_successrate[(session_successrate['Day'] == 'FOL') &
                                          (session_successrate['Trials Since Loom'] < 6)],
                  x = 'Trials Since Loom',
                    y = 'Success Rate',
                  hue = 'Cell Type',
                  palette = ct_cmap,
                  hue_order = celltypes,
                  s = 6,
                  linewidth = 1,
                  edgecolor = 'white',
                  alpha = 0.8, 
                )

plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Cell Type', 
           title_fontsize = 8,
           fontsize = 8)

sns.despine(top = True, right = True)

#plt.savefig('/Users/ryankast/Documents/FengLab/Meetings/LabMeetings/FLM14/SuccessRate_TrialsPastLoomCellTypeColors.svg',bbox_inches = 'tight')

In [ ]:
plt.figure(figsize = (6,3), dpi = 300)
plt.ylim(-0.1,1.1)




ax = sns.lineplot(data=session_successrate[(session_successrate['Day'] == 'FOL') &
                                          (session_successrate['Trials Since Loom'] < 6)],
                  x = 'Trials Since Loom',
                  y = 'Success Rate',
                  hue = 'Cell Type',
                  palette = ct_cmap,
                  hue_order = celltypes,
                  alpha = 0.8, 
                )

plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Cell Type', 
           title_fontsize = 8,
           fontsize = 8)

sns.despine(top = True, right = True)

##plt.savefig('/Users/ryankast/Documents/FengLab/Meetings/LabMeetings/FLM14/SuccessRate_TrialsPastLoomCellTypeColors.svg',bbox_inches = 'tight')

In [ ]:
sns.set_theme(style = 'white', 
              font = 'arial',
              font_scale = 2)

In [ ]:
mice = list(genotype_dict.keys())

colors = ['dimgrey',
          'dimgrey',
          'dimgrey',
          'dimgrey',
          'yellow',
          'yellow',
          'yellow',
          'darkturquoise',
          'magenta',
          'darkturquoise',
          'magenta',
          'magenta',
          'darkturquoise',
          'magenta',
          'darkturquoise',
          ]


micedict = dict(zip(mice, colors))
mouse_cmap = [micedict[x] for x in mice]

In [ ]:
ax = sns.PairGrid(data=session_successrate[((session_successrate['Day'] == 'FOL')) &
                                          (session_successrate['Trials Since Loom'] <= 0)],
                  x_vars = 'Trials Since Loom',
                  y_vars = 'Success Rate',
                  hue ='Mouse',
                  hue_order = mice,
                  height = 5,
                  palette = mouse_cmap,
                  )




ax.map(sns.pointplot, scale=1.5, errwidth=4)
ax.set(ylim=(0, 1.1),
       xticklabels = ['No Loom', 'Loom'],
       xlabel = 'Trial Type')


#sns.despine(fig=ax.fig, left=True)
#plt.savefig('/Users/ryankast/Documents/FengLab/Meetings/LabMeetings/FLM14/SuccessRate_LoomVsNoLoomPairGrid.svg', dpi =150)

In [ ]:
sns.set_theme(style = 'white', 
              font = 'arial',
              font_scale = 1)
plt.figure(figsize = (2,2.25), dpi = 150)


sns.barplot(data=session_successrate[((session_successrate['Day'] == 'FOL')) &
                                    (session_successrate['Trials Since Loom'] <= 0)],
           x = 'Trials Since Loom',
           y = 'Success Rate',
           color = 'lightgrey',
           width = 0.5)

plt.ylim(0, 1.1)

plt.yticks([0,0.2,0.4,0.6,0.8,1.0])

sns.despine(top = True,
            right = True)

##plt.savefig('/Users/ryankast/Documents/FengLab/Meetings/LabMeetings/FLM14/SuccessRate_LoomVsNoLoomBarPlot.svg', bbox_inches = 'tight')

In [ ]:
sns.set_theme(style = 'white', 
              font = 'arial',
              font_scale = 2)

afig = plt.figure(figsize = (6,3), dpi =300)

#mice = list(genotype_dict.keys())

colors = ['dimgrey',
          'dimgrey',
          'dimgrey',
          'dimgrey',
          'yellow',
          'yellow',
          'yellow',
          'darkturquoise',
          'magenta',
          'darkturquoise',
          'magenta',
          'magenta',
          'darkturquoise',
          'magenta',
          'darkturquoise',
          ]


#micedict = dict(zip(mice, colors))
#mouse_cmap = [micedict[x] for x in mice]


ax = sns.PairGrid(data=session_successrate[(session_successrate['Trials Since Loom'] == -1)],
                  x_vars = 'Day',
                  y_vars = 'Success Rate',
                  hue ='Mouse',
                  #hue_order = mice,
                  height = 5,
                  #palette = mouse_cmap,
                  )


ax.map(sns.pointplot, scale=1, errwidth=4)
ax.set(ylim=(0, 1.1),
       xlabel = '')


#sns.despine(fig=ax.fig, left=True)
#plt.savefig('/Users/ryankast/Documents/FengLab/Meetings/LabMeetings/FLM14/SuccessRate_Days2-5.png', dpi = 150)

In [ ]:
sns.set_theme(style = 'white', 
              font = 'arial',
              font_scale = 1)
plt.figure(figsize = (6,3), dpi = 300)
plt.ylim(-0.1,1.1)



ax = sns.lineplot(data=session_successrate[((session_successrate['Day'] == 'FOL')) &
                                          (session_successrate['Trials Since Loom'] <= 0)],
                  x = 'Trials Since Loom',
                  y = 'Success Rate',
                  hue ='Mouse',
                  markers = 'o',
                  markersize = 10,
                  linewidth = 2,)
                  #hue_order = celltypes,
                  #palette = ct_cmap)


plt.legend(bbox_to_anchor=(1.05, 1),
           loc='upper left', 
           borderaxespad=0, 
           title = 'Mouse', 
           title_fontsize = 8,
           fontsize = 8)

In [ ]:
graph = sns.FacetGrid(data = trialdf[trialdf['Active Port'] != 0],
                col = 'Day',
                row = 'Mouse',
                hue = 'Loom Trial',
                palette = )

graph.map(plt.scatter, 'Trial Number',
                'Digitized Trial Outcome')

In [ ]:
dfFullSet.columns.tolist()

In [ ]:
plt.figure(figsize=(3,3), dpi = 300)

sns.set_theme(style = 'white', 
              font = 'arial',
              font_scale = 1)
plt.figure(figsize = (6,3), dpi = 300)
plt.ylim(-0.1,1.1)



ax = sns.lineplot(data= ],
                  x = 'Trials Since Loom',
                  y = 'Success Rate',
                  hue ='Mouse',
                  markers = 'o',
                  markersize = 10,
                  linewidth = 2,)
                  #hue_order = celltypes,
                  #palette = ct_cmap)


plt.legend(bbox_to_anchor=(1.05, 1),
           loc='upper left', 
           borderaxespad=0, 
           title = 'Mouse', 
           title_fontsize = 8,
           fontsize = 8)

In [ ]:
cmap

In [ ]:
mouse = 'JAX-11M-1R'
day = 'FOL'

cmap = plt.get_cmap('cool_r')
cmap.set_under('black')  # Color for values less than vmin

# Very small float such that 0.0 != 0 + eps
eps = np.spacing(0.0)

DATAZ = dfFullSet[(dfFullSet['Mouse'] == mouse) & (dfFullSet['Day'] == day)]
norm = plt.Normalize(vmin=0,vmax=DATAZ['Trials Since Loom'].max())   

sns.scatterplot(data=DATAZ,
                s=50,
                x='Trial Number',
                y='Digitized Trial Outcome',
                palette = cmap,
                hue = 'Trials Since Loom')

In [ ]:
sns.set_theme(style = 'white', 
              font = 'arial',
              font_scale = 1)


for i in dfFullSet['Day'].unique().tolist():
    for j in dfFullSet['Mouse'].unique().tolist():
        plt.figure(figsize = (6,2), dpi =300)
        mouse = j
        day = i

        # Get the 'cool_r' colormap
        cmap = plt.get_cmap('cool')

        # Create a custom colormap that maps negative values to black and uses 'cool_r' for positive values
        colors = [(0.5, 0.5, 0.5)] + [cmap(i) for i in np.linspace(0, 1, 256)]

        custom_cmap = plt.cm.colors.ListedColormap(colors)

        DATAZ = trialdf[(trialdf['Mouse'] == mouse) & (trialdf['Day'] == day)]


        if len(DATAZ['Trials Since Loom'].unique().tolist()) > 1:
            labelnames = ['No Loom', 'Loom Trial']

        else:
            labelnames = ['No Loom']

        ax = sns.scatterplot(data=DATAZ,
                        s=40,
                        x='Trial Number',
                        y=DATAZ['Successful Response Time'] / 1000,
                        palette=custom_cmap,
                        hue='Trials Since Loom')

        plt.legend(
                   bbox_to_anchor=(1.05, 1), 
                   loc='upper left', 
                   borderaxespad=0, 
                   title = 'Trials Since Loom', 
                   title_fontsize = 8,
                   fontsize = 8)

        sns.despine(right=True,top =True)

        # Set y-axis tick labels
        #ytick_labels = ['Failure', 'Success']

        # Set y-axis tick positions and labels
        ax.set_yticks([0,30,60,90,120])
        ax.set_ylabel('Success\nResponse Time (s)')
        ax.set_ylim(-2,120)
        #ax.set_yticklabels(ytick_labels)
        ax.set_title(mouse + "\n" + day + " Session",pad=10)

        sns.despine(top = True, right = True)
        #plt.savefig('/Users/ryankast/Documents/FengLab/Meetings/LabMeetings/FLM14/SessionResponseTimeData/' + mouse + ' ' + day + '.svg',bbox_inches = 'tight')


In [ ]:
plt.figure(figsize = (6,2), dpi =300)
mouse = 'JAX-16M-2L'
day = 'Day4'

# Get the 'cool_r' colormap
cmap = plt.get_cmap('cool')

# Create a custom colormap that maps negative values to black and uses 'cool_r' for positive values
colors = [(0.5, 0.5, 0.5)] + [cmap(i) for i in np.linspace(0, 1, 256)]

custom_cmap = plt.cm.colors.ListedColormap(colors)

DATAZ = trialdf[(trialdf['Mouse'] == mouse) & (trialdf['Day'] == day)]


if len(DATAZ['Trials Since Loom'].unique().tolist()) > 1:
    labelnames = ['No Loom', 'Loom Trial']
    
else:
    labelnames = ['No Loom']

ax = sns.scatterplot(data=DATAZ,
                s=40,
                x='Trial Number',
                y=DATAZ['Digitized Trial Outcome'],
                palette=custom_cmap,
                hue='Trials Since Loom')

plt.legend(
           bbox_to_anchor=(1.05, 1), 
           loc='upper left', 
           borderaxespad=0, 
           title = 'Trials Since Loom', 
           title_fontsize = 8,
           fontsize = 8)

#Set y-axis tick labels
ytick_labels = ['Failure', 'Success']

# Set y-axis tick positions and labels
ax.set_yticks([0, 1])
ax.set_ylabel('Trial Outcome')
ax.set_yticklabels(ytick_labels)
ax.set_title(mouse)
plt.show()